# Imports and file details

In [1]:
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()
sns.set_style("ticks")

import os
import sys
sys.path.insert(0,'..')

import pandas as pd
import json
import numpy as np

import dask.dataframe as dd
from dask.dataframe.utils import make_meta

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

from src_new.utils import SQLParserSchema, PGLastSchema, get_file_encodings, SQLGlotSchema, SimpleDDLParserSchema,TIDBMysqlSchema,JSqlParserSchema, RustParserSchema

from dask.diagnostics import ProgressBar
ProgressBar().register()

C:\Users\geacu\AppData\Local\Temp\ipykernel_16092\2258049444.py:27: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


In [2]:
def read_partial(parser, schema_type, cols):
    _outdir = '../out_new/'

    if parser == 'sqlparser':
        p = SQLParserSchema()
        _outdir += 'sqlparser'
    elif parser == 'pglast':
        p = PGLastSchema()
        _outdir += 'pglast'
    elif parser == 'sqlglot':
        p = SQLGlotSchema()
        _outdir += 'sqlglot'
    elif parser == 'simple':
        p = SimpleDDLParserSchema()
        _outdir += 'simpleddlparser'
    elif parser == 'tidb':
        p = TIDBMysqlSchema()
        _outdir += 'tidb_mysql'
    elif parser == 'jsqlparser':
        p = JSqlParserSchema()
        _outdir += 'jsqlparser'
    elif parser == 'rustparser':
        p = RustParserSchema()
        _outdir += 'rustparser'
    
    if schema_type == 'file':
        _schema = p.file_level_schema
        _outdir += '/'
    elif schema_type == 'stmt':
        _schema = p.statement_list_sch
        _outdir += '_details/'
    
    ddf = dd.read_parquet(_outdir,columns=cols, schema=_schema, split_row_groups=True, calculate_divisions=True,engine='pyarrow')
    return ddf

In [3]:
# Read file details
fd = []
for file in os.listdir('../out_new/filedetails/'):
        full_filename = "%s/%s" % ('../out_new/filedetails/', file)
        with open(full_filename,'r') as fi:
            dict = json.load(fi)
            for item in dict:
                  fd.append(item)

filedetails_df = pd.DataFrame.from_dict(fd)
del fd

In [4]:
rust_ddf = read_partial('rustparser','file',RustParserSchema().file_level_schema.names)
rust_details_ddf = read_partial('rustparser','stmt',RustParserSchema().statement_list_sch.names)


In [ ]:
rust_ddf.groupby('parsed')['file_id'].count().compute()

# PGlast match

In [ ]:
rust_ddf[rust_ddf.parsed == 1].head()

In [ ]:
not_parsed = rust_ddf[rust_ddf.parsed == 0]

In [ ]:
not_parsed = not_parsed[not_parsed.parse_error.str.contains("NON_PARSER_ERROR|",regex=False)]

In [ ]:
not_parsed['file_id'].count().compute()

In [ ]:
# schema prep
join_schema = pa.schema([])
join_schema = join_schema.append(pa.field("file_id",pa.string()))

# pglast prep

pglast_ddf = read_partial('pglast','file',PGLastSchema().file_level_schema.names)

import ast
pglast_ddf['x'] = pglast_ddf.apply(
    lambda row:
    #'a',
    [] if row['counter_str'] is None else [item for item in ast.literal_eval(row['counter_str'] or 'None') if 'CreateSeqStmt' in item],
    axis=1,
    meta=('x', 'string'))

pglast_ddf['num_create_seq'] = pglast_ddf.apply(
    lambda row:
    row['x'][0]['CreateSeqStmt'] if len(row['x']) > 0 else 0,
    axis=1,
    meta=('num_create_seq', 'float'))

pglast_ddf = pglast_ddf.drop(columns=['x'])  

pglast_ddf['num_tables_without_create_seq'] = pglast_ddf['num_distinct_tables'] - pglast_ddf['num_create_seq']

pglast_ddf['table_list_str'] = pglast_ddf.apply(
    lambda row:
    "|".join(str(x) for x in sorted(row['table_list'])) if row['table_list'] is not None else None,
    axis=1,
    meta=('table_list_str', 'string'))
pglast_ddf['column_list_str'] = pglast_ddf.apply(
    lambda row:
    "|".join(str(x) for x in sorted(row['columns_list'])) if row['columns_list'] is not None else None,
    axis=1,
    meta=('column_list_str', 'string'))
pglast_ddf['schema_list_str'] = pglast_ddf.apply(
    lambda row:
    "|".join(str(x) for x in sorted(row['schema_list'])) if row['schema_list'] is not None else None,
    axis=1,
    meta=('schema_list_str', 'string'))
pglast_ddf['view_list_str'] = pglast_ddf.apply(
    lambda row:
    "|".join(str(x) for x in sorted(row['view_list'])) if row['view_list'] is not None else None,
    axis=1,
    meta=('view_list_str', 'string'))
pglast_ddf['db_list_str'] = pglast_ddf.apply(
    lambda row:
    "|".join(str(x) for x in sorted(row['db_list'])) if row['db_list'] is not None else None,
    axis=1,
    meta=('db_list_str', 'string'))

try:
    del dict
except:
    pass
pglast_ddf = pglast_ddf.rename(columns=dict(zip(pglast_ddf.columns, [item.replace("pglast_file_id","file_id") for item in ["pglast_" + item for item in pglast_ddf.columns]])))

for item in PGLastSchema().file_level_schema:
    if item.name != 'file_id':
        join_schema = join_schema.append(pa.field("pglast_" + item.name,item.type))
join_schema = join_schema.append(pa.field("pglast_num_create_seq", pa.float64()))
join_schema = join_schema.append(pa.field("pglast_num_tables_without_create_seq", pa.float64()))

join_schema = join_schema.append(pa.field("pglast_table_list_str", pa.string()))
join_schema = join_schema.append(pa.field("pglast_column_list_str", pa.string()))
join_schema = join_schema.append(pa.field("pglast_schema_list_str", pa.string()))
join_schema = join_schema.append(pa.field("pglast_view_list_str", pa.string()))
join_schema = join_schema.append(pa.field("pglast_db_list_str", pa.string()))

rust_ddf = read_partial('rustparser','file',RustParserSchema().file_level_schema.names)

rust_ddf['table_list_str'] = rust_ddf.apply(
    lambda row:
    "|".join(str(x) for x in sorted(row['table_list'])) if row['table_list'] is not None else None,
    axis=1,
    meta=('table_list_str', 'string'))
rust_ddf['column_list_str'] = rust_ddf.apply(
    lambda row:
    "|".join(str(x) for x in sorted(row['column_list'])) if row['column_list'] is not None else None,
    axis=1,
    meta=('column_list_str', 'string'))
rust_ddf['schema_list_str'] = rust_ddf.apply(
    lambda row:
    "|".join(str(x) for x in sorted(row['schema_list'])) if row['schema_list'] is not None else None,
    axis=1,
    meta=('schema_list_str', 'string'))
rust_ddf['view_list_str'] = rust_ddf.apply(
    lambda row:
    "|".join(str(x) for x in sorted(row['view_list'])) if row['view_list'] is not None else None,
    axis=1,
    meta=('view_list_str', 'string'))
rust_ddf['db_list_str'] = rust_ddf.apply(
    lambda row:
    "|".join(str(x) for x in sorted(row['db_list'])) if row['db_list'] is not None else None,
    axis=1,
    meta=('db_list_str', 'string'))

# renaming cols
jsql_ddf = rust_ddf.rename(columns=dict(zip(rust_ddf.columns, [item.replace("rustparser_file_id","file_id") for item in ["rustparser_" + item for item in rust_ddf.columns]])))

for item in JSqlParserSchema().file_level_schema:
    if item.name != 'file_id':
        join_schema = join_schema.append(pa.field("jsqlparser_" + item.name,item.type))

join_schema = join_schema.append(pa.field("rustparser_table_list_str", pa.string()))
join_schema = join_schema.append(pa.field("rustparser_column_list_str", pa.string()))
join_schema = join_schema.append(pa.field("rustparser_schema_list_str", pa.string()))
join_schema = join_schema.append(pa.field("rustparser_view_list_str", pa.string()))
join_schema = join_schema.append(pa.field("rustparser_db_list_str", pa.string()))

pglast_joined_jsql= dd.merge(
    pglast_ddf,
    jsql_ddf,
    on=['file_id'],
    how='inner',indicator=False,suffixes=['_pglast','_rustparser'])

In [ ]:
parsed_both = pglast_joined_jsql[(pglast_joined_jsql.pglast_parsed_file == 1) & (pglast_joined_jsql.rustparser_parsed == 1)]

In [ ]:
parsed_both['file_id'].count().compute()
# 3163

In [ ]:
parsed_both[parsed_both['pglast_num_ctr_foreign'] == parsed_both['rustparser_num_ctr_foreign']]['file_id'].count().compute()

In [ ]:
parsed_both[parsed_both['pglast_num_distinct_columns'] == parsed_both['rustparser_num_distinct_columns']]['file_id'].count().compute()

In [ ]:
parsed_both[(parsed_both['pglast_num_distinct_tables'] != parsed_both['rustparser_num_distinct_tables'])]['file_id'].count().compute()

In [ ]:
parsed_both[(parsed_both['pglast_num_distinct_tables'] != parsed_both['rustparser_num_distinct_tables']) &
(parsed_both.pglast_num_view_stmt == 0)][['file_id','pglast_table_list','pglast_num_distinct_tables','rustparser_num_distinct_tables', 'rustparser_table_list']].head()

In [ ]:
parsed_both[parsed_both['pglast_num_ctr_primary'] == parsed_both['rustparser_num_ctr_primary']]['file_id'].count().compute()

In [ ]:
parsed_both[parsed_both['pglast_num_ctr_unique'] == parsed_both['rustparser_num_ctr_unique']]['file_id'].count().compute()

In [ ]:
parsed_both[parsed_both['pglast_num_ctr_notnull'] == parsed_both['rustparser_num_ctr_notnull']]['file_id'].count().compute()

In [ ]:
parsed_both[parsed_both['pglast_num_distinct_schemas'] == parsed_both['rustparser_num_distinct_schemas']]['file_id'].count().compute()

# Error analysis

In [ ]:
rust_ddf = read_partial('rustparser','file',RustParserSchema().file_level_schema.names)
rust_details_ddf = read_partial('rustparser','stmt',RustParserSchema().statement_list_sch.names)

## File level

In [ ]:
rust_ddf.groupby('parsed')['file_id'].count().compute()

In [ ]:
rust_details_ddf.groupby('parsed')['file_id'].count().compute()

In [ ]:
not_parsed = rust_ddf[rust_ddf.parsed == 0]
not_parsed.head()

# New error analysis - stmt level -> REDOING with limited list of dialects for unparsed files + making sure that mysql/tsql are at end to capture actual error

## Read and Prep

In [4]:
rust_details_ddf = dd.read_parquet('../out_new/rustparserforerrors_details/',columns=RustParserSchema().statement_list_sch.names, schema=RustParserSchema().statement_list_sch, split_row_groups=True, calculate_divisions=True,engine='pyarrow')
rust_details_ddf.groupby("parsed")['parsed'].count().compute(scheduler='processes')

# initially!
# parsed
# 0    4246949 
# 1    20380557

[########################################] | 100% Completed | 4.10 ss


parsed
0    4386386 
1    20241120
Name: parsed, dtype: int64

In [5]:
rust_details_ddf['file_id'].nunique().compute(scheduler='processes')

[########################################] | 100% Completed | 4.36 ss


373156

In [6]:
parsed = rust_details_ddf[rust_details_ddf.parsed == 1]
parsed['file_id'].nunique().compute(scheduler='processes')

[########################################] | 100% Completed | 4.47 ss


327368

In [7]:
not_parsed = rust_details_ddf[rust_details_ddf.parsed == 0]
not_parsed['file_id'].nunique().compute(scheduler='processes')

[########################################] | 100% Completed | 4.18 ss


280024

In [8]:
not_parsed[not_parsed.parse_error.str.contains('Query parsing failed.\n\tsql parser error:',regex=False,case=False)]['file_id'].count().compute(scheduler='processes')
# 6 missing, f it (initially)
# after: 14 missing, f it

[########################################] | 100% Completed | 29.89 s


4386372

In [9]:
not_parsed_split_1 = not_parsed[not_parsed.parse_error.str.contains('Query parsing failed.\n\tsql parser error:',regex=False,case=False)]
not_parsed_split_1['split_err'] = not_parsed_split_1.apply(
    lambda row:
    row['parse_error'].split('Query parsing failed.\n\tsql parser error:')[1],
    axis=1,
    meta=('split_err', 'string'))

In [ ]:
not_parsed_split_1[
    (not_parsed_split_1.parse_error.str.contains('Expected',regex=False,case=False)) &
    (not_parsed_split_1.parse_error.str.contains('found',regex=False,case=False))
]['file_id'].count().compute(scheduler='processes')

# 11,898 missing (initially)

# 10 188 with Unterminated

In [10]:
not_parsed_split_1['expected'] = not_parsed_split_1.apply(
    lambda row:
    row['split_err'].split('Expected')[1].split('found:')[0].strip() if 'Expected' in row['split_err'] and 'found' in row['split_err'] else None,
    axis=1,
    meta=('trunc_err', 'string'))

not_parsed_split_1['found'] = not_parsed_split_1.apply(
    lambda row:
    row['split_err'].split('Expected')[1].split('found:')[1].strip() if 'Expected' in row['split_err'] and 'found' in row['split_err'] else None,
    axis=1,
    meta=('trunc_err', 'string'))

not_parsed_split_1['unterminated_what'] = not_parsed_split_1.apply(
    lambda row:
    row['split_err'].split('Unterminated')[1].split('at')[0].strip() if 'Unterminated' in row['split_err'] and 'at' in row['split_err'] else None,
    axis=1,
    meta=('trunc_err', 'string'))

not_parsed_split_1['unterminated_where'] = not_parsed_split_1.apply(
    lambda row:
    row['split_err'].split('Unterminated')[1].split('at')[1].strip() if 'Unterminated' in row['split_err'] and 'at' in row['split_err'] else None,
    axis=1,
    meta=('trunc_err', 'string'))

In [11]:
not_parsed_schema = pa.schema([])
not_parsed_schema = not_parsed_schema.append(pa.field("file_id",pa.string()))
not_parsed_schema = not_parsed_schema.append(pa.field("statement_id",pa.int64()))
not_parsed_schema = not_parsed_schema.append(pa.field("parse_error",pa.string()))
not_parsed_schema = not_parsed_schema.append(pa.field("expected",pa.string()))
not_parsed_schema = not_parsed_schema.append(pa.field("found",pa.string()))
not_parsed_schema = not_parsed_schema.append(pa.field("unterminated_what",pa.string()))
not_parsed_schema = not_parsed_schema.append(pa.field("unterminated_where",pa.string()))
not_parsed_schema = not_parsed_schema.append(pa.field("original",pa.string()))

In [12]:
#not_parsed_split_1.to_parquet('../out_new/rustparser_details_unparsed_stmts_only/',schema=not_parsed_schema,overwrite=True)

[########################################] | 100% Completed | 254.36 s


## Read only unparsed statements -> with errors split + to_write_down_hard start

In [13]:
not_parsed_schema = pa.schema([])
not_parsed_schema = not_parsed_schema.append(pa.field("file_id",pa.string()))
not_parsed_schema = not_parsed_schema.append(pa.field("statement_id",pa.int64()))
not_parsed_schema = not_parsed_schema.append(pa.field("parse_error",pa.string()))
not_parsed_schema = not_parsed_schema.append(pa.field("expected",pa.string()))
not_parsed_schema = not_parsed_schema.append(pa.field("found",pa.string()))
not_parsed_schema = not_parsed_schema.append(pa.field("unterminated_what",pa.string()))
not_parsed_schema = not_parsed_schema.append(pa.field("unterminated_where",pa.string()))
not_parsed_schema = not_parsed_schema.append(pa.field("original",pa.string()))

In [21]:
def get_stmt_and_file_count(dask_df):
    x = dask_df['file_id'].nunique().compute(scheduler='processes')
    y = dask_df['file_id'].count().compute(scheduler='processes')
    return y,x

In [ ]:
# unparsed 4246949; -6 from start
# 11,898 without expected/found => set to None
# out of which 10188 Unterminated

In [14]:
unparsed = dd.read_parquet('../out_new/rustparser_details_unparsed_stmts_only/', columns = not_parsed_schema.names ,schema=not_parsed_schema, split_row_groups=True, calculate_divisions=True,engine='pyarrow')

In [15]:
expected_only = unparsed[~unparsed.expected.isnull()]

In [16]:
unterminated_only = unparsed[~unparsed.unterminated_what.isnull()]

In [282]:
to_write_down_hard = []
to_write_down_hard.append(unparsed[unparsed.expected.isnull()].compute(scheduler='processes'))

[########################################] | 100% Completed | 6.05 ss


## Unterminated only -> mostly caused by comments, skipping

In [154]:
unterminated_only.head()

[########################################] | 100% Completed | 246.61 ms


file_id  statement_id                                                                                      parse_error expected found unterminated_what     unterminated_where                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [161]:
#to_write_down_hard.append(unterminated_only.compute(scheduler='processes'))

[########################################] | 100% Completed | 6.71 ss


## Expected only

### Keywords and remember those that you're not touching to write down

In [283]:
to_write = expected_only[
    (expected_only.expected != 'an SQL statement,') &
    (expected_only.expected != 'end of statement,') &
    (expected_only.expected != 'ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,') &
    (expected_only.expected != 'a list of columns in parentheses,') &
    (expected_only.expected != "',' or ')' after column definition,") &
    (expected_only.expected != "),") &
    (expected_only.expected != "an object type after CREATE,") &
    (expected_only.expected != "SELECT, VALUES, or a subquery in the query body,") &
    (expected_only.expected != "TABLE, VIEW, INDEX, ROLE, SCHEMA, FUNCTION, STAGE or SEQUENCE after DROP,") &
    (expected_only.expected != "one of TABLE or INDEX,") &
    (expected_only.expected != "identifier,") &
    (expected_only.expected != "PRIMARY, UNIQUE, FOREIGN, or CHECK,") &
    (expected_only.expected != "ON,") &
    (expected_only.expected != "SET/DROP NOT NULL, SET DEFAULT, SET DATA TYPE after ALTER COLUMN,")
]
to_write.count().compute(scheduler='processes')

[########################################] | 100% Completed | 7.01 ss


file_id               92856
statement_id          92856
parse_error           92856
expected              92856
found                 92856
unterminated_what     0    
unterminated_where    0    
original              92856
dtype: int64

In [284]:
to_write_down_hard.append(to_write.compute(scheduler='processes'))

[########################################] | 100% Completed | 7.25 ss


In [182]:
print(get_stmt_and_file_count(no_after))

[########################################] | 100% Completed | 9.04 ss
[########################################] | 100% Completed | 11.20 s
(306404, 50604)


In [135]:
expected_only['expected'].value_counts().compute(scheduler='processes')

[########################################] | 100% Completed | 3.71 ss


an SQL statement,                                                               1503548
end of statement,                                                               1257021
ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,                         417369 
a list of columns in parentheses,                                               345329 
',' or ')' after column definition,                                             329127 
),                                                                              142034 
an object type after CREATE,                                                    76579  
SELECT, VALUES, or a subquery in the query body,                                70698  
TABLE, VIEW, INDEX, ROLE, SCHEMA, FUNCTION, STAGE or SEQUENCE after DROP,       44497  
one of TABLE or INDEX,                                                          39405  
identifier,                                                                     38696  
equals sign or TO,              

In [18]:
expected_only['found'].value_counts().compute(scheduler='processes')

[########################################] | 100% Completed | 3.49 ss


UNLOCK                            450055
LOCK                              448751
AUTO_INCREMENT                    418326
,                                 325621
MODIFY                            305216
                                   ...  
'DROP SEQUENCE address_id_seq'    1     
`country_`                        1     
`countries`                       1     
'DROP SEQUENCE admin_id_seq'      1     
🤨                                 1     
Name: found, Length: 39113, dtype: int64

### Expected an SQL statement

In [20]:
expected_sql_statement = expected_only[expected_only.expected == 'an SQL statement,']
expected_sql_statement['found'].value_counts().compute(scheduler='processes')

[########################################] | 100% Completed | 6.91 ss


UNLOCK       450048
LOCK         448643
#            141082
DELIMITER    72973 
GO           66178 
             ...   
26058        1     
261          1     
262145       1     
2628         1     
🤨            1     
Name: found, Length: 14993, dtype: int64

In [58]:
print(get_stmt_and_file_count(expected_sql_statement))

[########################################] | 100% Completed | 8.03 ss
[########################################] | 100% Completed | 9.06 ss
(1503548, 89174)


In [49]:
no_go['found'].value_counts().compute(scheduler='processes')

[########################################] | 100% Completed | 8.87 ss


REPLACE                                                                             23759
END                                                                                 18361
\                                                                                   17634
################################################################################    17303
/                                                                                   13910
                                                                                    ...  
26058                                                                               1    
261                                                                                 1    
262145                                                                              1    
2628                                                                                1    
🤨                                                                                   1    
Name: foun

#### UNLOCK - LOCK

In [24]:
unlock_check = expected_sql_statement[
    (expected_sql_statement.found.str.contains('UNLOCK',regex=False,case=False)) &
    (
        (expected_sql_statement.original.str.contains('TABLES',regex=False,case=False)) |
        (expected_sql_statement.original.str.contains('TABLE',regex=False,case=False))
    )
]

print(get_stmt_and_file_count(unlock_check))

[########################################] | 100% Completed | 7.16 ss
[########################################] | 100% Completed | 9.95 ss
(450241, 46578)


In [23]:
unlock_check.head()

[########################################] | 100% Completed | 309.09 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
89,014766,21,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: UNLOCK","an SQL statement,",UNLOCK,None,None,UNLOCK TABLES;
268,014815,19,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: UNLOCK","an SQL statement,",UNLOCK,None,None,UNLOCK TABLES;
270,014815,28,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: UNLOCK","an SQL statement,",UNLOCK,None,None,UNLOCK TABLES;
272,014815,37,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: UNLOCK","an SQL statement,",UNLOCK,None,None,UNLOCK TABLES;
274,014815,46,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: UNLOCK","an SQL statement,",UNLOCK,None,None,UNLOCK TABLES;


In [27]:
no_unlock = expected_sql_statement[
    ~((expected_sql_statement.found.str.contains('UNLOCK',regex=False,case=False)) &
    (
        (expected_sql_statement.original.str.contains('TABLES',regex=False,case=False)) |
        (expected_sql_statement.original.str.contains('TABLE',regex=False,case=False))
    ))
]

In [28]:
lock_check = no_unlock[
    (no_unlock.found.str.contains('LOCK',regex=False,case=False)) &
    (
        (no_unlock.original.str.contains('TABLES',regex=False,case=False)) |
        (no_unlock.original.str.contains('TABLE',regex=False,case=False))
    )
]

print(get_stmt_and_file_count(lock_check))

[########################################] | 100% Completed | 9.41 ss
[########################################] | 100% Completed | 9.90 ss
(448901, 46454)


In [29]:
lock_check.head()

[########################################] | 100% Completed | 377.42 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
88,014766,17,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: LOCK","an SQL statement,",LOCK,None,None,--\n-- Dumping data for table `movie`\n--\n\nLOCK TABLES `movie` WRITE;
267,014815,15,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: LOCK","an SQL statement,",LOCK,None,None,--\n-- Dumping data for table `wp_tfk_item`\n--\n\nLOCK TABLES `wp_tfk_item` WRITE;
269,014815,25,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: LOCK","an SQL statement,",LOCK,None,None,--\n-- Dumping data for table `wp_tfk_item_comment`\n--\n\nLOCK TABLES `wp_tfk_item_comment` WRITE;
271,014815,34,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: LOCK","an SQL statement,",LOCK,None,None,--\n-- Dumping data for table `wp_tfk_item_comment_like`\n--\n\nLOCK TABLES `wp_tfk_item_comment_like` WRITE;
273,014815,43,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: LOCK","an SQL statement,",LOCK,None,None,--\n-- Dumping data for table `wp_tfk_item_file`\n--\n\nLOCK TABLES `wp_tfk_item_file` WRITE;


#### hashtag

In [30]:
no_lock_unlock = no_unlock[
    ~((no_unlock.found.str.contains('LOCK',regex=False,case=False)) &
    (
        (no_unlock.original.str.contains('TABLES',regex=False,case=False)) |
        (no_unlock.original.str.contains('TABLE',regex=False,case=False))
    ))
]

In [35]:
hashtag_check = no_lock_unlock[
    (no_lock_unlock.found == '#')
]

print(get_stmt_and_file_count(hashtag_check))

[########################################] | 100% Completed | 9.53 ss
[########################################] | 100% Completed | 12.86 s
(141082, 10663)


In [33]:
hashtag_check.head()

[########################################] | 100% Completed | 222.79 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
1,014744,0,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: #","an SQL statement,",#,None,None,"# By: Ron Cordell - roncordell\n# I didn't see this anywhere, so I thought I'd post it here. This is the script from Quartz to create the tables in a MySQL database, modified to use INNODB instead of MYISAM.\n\n\n# make sure you have UTF-8 collaction for best .NET interoperability\n# CREATE DATABASE quartznet CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;\n\nDROP TABLE IF EXISTS QRTZ_FIRED_TRIGGERS;"
566,014847,0,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: #","an SQL statement,",#,None,None,"# This is a fix for InnoDB in MySQL >= 4.1.x\n# It ""suspends judgement"" for fkey relationships until are tables are set.\nSET FOREIGN_KEY_CHECKS = 0;"
567,014847,1,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: #","an SQL statement,",#,None,None,#-----------------------------------------------------------------------------\n#-- wf_constant\n#-----------------------------------------------------------------------------\n\nDROP TABLE IF EXISTS `wf_constant`;
573,014847,7,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: #","an SQL statement,",#,None,None,"# This restores the fkey checks, after having unset them earlier\nSET FOREIGN_KEY_CHECKS = 1;"
578,014851,0,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: #","an SQL statement,",#,None,None,"# Ensembl core table definitions\n#\n\n# Conventions:\n# - use lower case and underscores\n# - internal ids are integers named tablename_id\n# - same name is given in foreign key relations\n\n\n/**\n@header Fundamental Tables\n@desc A PDF document of the schema is available <a href=""fundamental_tables_core.pdf"">here</a>.\n\n*/\n\n\n/**\n@table assembly\n@desc The assembly table states, which parts of seq_regions are exactly equal. It enables to transform coordinates between seq_regions.\nTypically this contains how chromosomes are made of contigs, clones out of contigs, and chromosomes out of supercontigs.\nIt allows you to artificially chunk chromosome sequence into smaller parts.\n\nThe data in this table defines the ""static golden path"", i.e. the best effort draft full genome sequence as determined by the UCSC or NCBI (depending which assembly you are using).\nEach row represents a component, e.g. a contig, (comp_seq_region_id, FK from seq_region table) at least part of which is present in the golden path.\nThe part of the component that is in the path is delimited by fields cmp_start and cmp_end (start < end), and the absolute position within the golden path chromosome (or other appropriate assembled structure) (asm_seq_region_id) is given by asm_start and asm_end.\n\n@column asm_seq_region_id Assembly sequence region id. Primary key, internal identifier. Foreign key references to the @link seq_region table.\n@column cmp_seq_region_id Component sequence region id. Foreign key references to the @link seq_region table.\n@column asm_start Start absolute position within the golden path chromosome.\n@column asm_end End absolute position within the golden path chromosome.\n@column cmp_start Component start position within the golden path chromosome.\n@column cmp_end Component start position within the golden path chromosome.\n@column ori Orientation: 1 - sense; -1 - antisense.\n\n\n@see seq_region\n@see supercontigs\n\n*/\n\n\nCREATE TABLE assembly (\n\n asm_seq_region_id INT(10) UNSIGNED NOT NULL,\n cmp_seq_region_id INT(10) UNSIGNED NOT NULL,\n asm_start INT(10) NOT NULL,\n asm_end INT(10) NOT NULL,\n cmp_start INT(10) NOT NULL,\n cmp_end INT(10) NOT NULL,\n ori TINYINT NOT NULL,\n\n KEY cmp_seq_region_idx (cmp_seq_region_id),\n KEY asm_seq_region_idx (asm_seq_region_id, asm_start),\n UNIQUE KEY all_idx (asm_seq_region_id, cmp_seq_region_id, asm_start, asm_

#### delimiter

In [36]:
no_hashtag = no_lock_unlock[
    ~(no_lock_unlock.found == '#')
]

In [40]:
delimiter_check = no_hashtag[
    (no_hashtag.found == 'DELIMITER')
]
print(get_stmt_and_file_count(delimiter_check))

[########################################] | 100% Completed | 7.90 ss
[########################################] | 100% Completed | 10.86 s
(72973, 8470)


In [39]:
delimiter_check['original'].value_counts().compute(scheduler='processes')

[########################################] | 100% Completed | 9.79 ss


DELIMITER ;                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              42767
DELIMITER %%;                                                                                                                                                                                                                            

In [43]:
#delimiter_check.head()
delimiter_check[delimiter_check.original.str.contains("CREATE",regex=False,case=False)].head(npartitions=-1)

[########################################] | 100% Completed | 11.84 s


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
909,014884,8,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: DELIMITER","an SQL statement,",DELIMITER,None,None,--\n-- Database: `gallery`\n--\n\nDELIMITER $$\n--\n-- Procedures\n--\nCREATE DEFINER=`root`@`localhost` PROCEDURE `exhibitiondetail` () SELECT * FROM exhibition$$\n\nDELIMITER ;
910,014884,15,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: DELIMITER","an SQL statement,",DELIMITER,None,None,--\n-- Triggers `login`\n--\nDELIMITER $$\nCREATE TRIGGER `toLower` BEFORE INSERT ON `login` FOR EACH ROW SET NEW.email=LOWER(NEW.email)\n$$\nDELIMITER ;
2368,015059,2,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: DELIMITER","an SQL statement,",DELIMITER,None,None,"DELIMITER //\nCREATE PROCEDURE MigrationsScript()\nBEGIN\n IF NOT EXISTS(SELECT 1 FROM `__EFMigrationsHistory` WHERE `MigrationId` = '20191201000001_InitialCreate') THEN\n\n CREATE TABLE `AspNetRoles` (\n `Id` int NOT NULL AUTO_INCREMENT,\n `Name` varchar(191) CHARACTER SET utf8mb4 NULL,\n `NormalizedName` varchar(191) CHARACTER SET utf8mb4 NULL,\n `ConcurrencyStamp` longtext CHARACTER SET utf8mb4 NULL,\n CONSTRAINT `PK_AspNetRoles` PRIMARY KEY (`Id`)\n );\n\n END IF;\nEND //\nDELIMITER ;"
2372,015059,6,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: DELIMITER","an SQL statement,",DELIMITER,None,None,"DELIMITER //\nCREATE PROCEDURE MigrationsScript()\nBEGIN\n IF NOT EXISTS(SELECT 1 FROM `__EFMigrationsHistory` WHERE `MigrationId` = '20191201000001_InitialCreate') THEN\n\n CREATE TABLE `AspNetUsers` (\n `Id` int NOT NULL AUTO_INCREMENT,\n `UserName` varchar(191) CHARACTER SET utf8mb4 NULL,\n `NormalizedUserName` varchar(191) CHARACTER SET utf8mb4 NULL,\n `Email` varchar(191) CHARACTER SET utf8mb4 NULL,\n `NormalizedEmail` varchar(191) CHARACTER SET utf8mb4 NULL,\n `EmailConfirmed` tinyint(1) NOT NULL,\n `PasswordHash` longtext CHARACTER SET utf8mb4 NULL,\n `SecurityStamp` longtext CHARACTER SET utf8mb4 NULL,\n `ConcurrencyStamp` longtext CHARACTER SET utf8mb4 NULL,\n `PhoneNumber` longtext CHARACTER SET utf8mb4 NULL,\n `PhoneNumberConfirmed` tinyint(1) NOT NULL,\n `TwoFactorEnabled` tinyint(1) NOT NULL,\n `LockoutEnd` datetime(6) NULL,\n `LockoutEnabled` tinyint(1) NOT NULL,\n `AccessFailedCount` int NOT NULL,\n `LastLogin` datetime(6) NULL,\n `CreatedAt` datetime(6) NULL,\n `UpdatedAt` datetime(6) NULL,\n CONSTRAINT `PK_AspNetUsers` PRIMARY KEY (`Id`)\n );\n\n END IF;\nEND //\nDELIMITER ;"
2376,015059,10,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: DELIMITER","an SQL statement,",DELIMITER,None,None,"DELIMITER //\nCREATE PROCEDURE MigrationsScript()\nBEGIN\n IF NOT EXISTS(SELECT 1 FROM `__EFMigrationsHistory` WHERE `MigrationId` = '20191201000001_InitialCreate') THEN\n\n CREATE TABLE `AspNetRoleClaims` (\n `Id` int NOT NULL AUTO_INCREMENT,\n `RoleId` int NOT NULL,\n `ClaimType` longtext CHARACTER SET utf8mb4 NULL,\n `ClaimValue` longtext CHARACTER SET utf8mb4 NULL,\n CONSTRAINT `PK_AspNetRoleClaims` PRIMARY KEY (`Id`),\n CONSTRAINT `FK_AspNetRoleClaims_AspNetRoles_RoleId` FOREIGN KEY (`RoleId`) REFERENCES `AspNetRoles` (`Id`) ON DELETE CASCADE\n );\n\n END IF;\nEND //\nDELIMITER ;"


#### GO

In [44]:
no_delimiter = no_hashtag[
    ~(no_hashtag.found == 'DELIMITER')
]

In [47]:
go_check = no_delimiter[
    (no_delimiter.found == 'GO')
]
print(get_stmt_and_file_count(go_check))

[########################################] | 100% Completed | 8.91 ss
[########################################] | 100% Completed | 10.02 s
(66178, 3200)


In [46]:
go_check.head()

[########################################] | 100% Completed | 268.53 ms


file_id  statement_id                                                                      parse_error           expected found unterminated_what unterminated_where                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

#### replace

In [48]:
no_go = no_delimiter[
    ~(no_delimiter.found == 'GO')
]

In [52]:
replace_check = no_go[
    (no_go.found == 'REPLACE') &
    (no_go.original.str.contains('REPLACE INTO',case=False,regex=False))
]
print(get_stmt_and_file_count(replace_check))

[########################################] | 100% Completed | 11.20 s
[########################################] | 100% Completed | 10.40 s
(23699, 617)


In [51]:
replace_check.head()

[########################################] | 100% Completed | 369.26 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
13327,016087,12,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: REPLACE","an SQL statement,",REPLACE,None,None,"REPLACE INTO `categories` (`categoryId`, `name`) VALUES\n\t(1, 'Drinks\r\n'),\n\t(2, 'Fruits'),\n\t(3, 'Meats, Eggs'),\n\t(4, 'Vegetables');"
13329,016087,16,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: REPLACE","an SQL statement,",REPLACE,None,None,"REPLACE INTO `products` (`productId`, `name`, `price`, `description`, `image`, `stock`, `categoryId`) VALUES\n\t(1, 'Wine', '200', 'White wine', 'wine01.jpg', 5, 1),\n\t(2, 'Wine', '500', 'Red wine', 'wine02.jpg', 1, 1),\n\t(3, 'Wine', '400', 'Fine wine', 'wine03.jpg', 3, 1),\n\t(4, 'Wine', '1000', 'Fine red wine', 'wine04.jpg', 2, 1),\n\t(5, 'Wine', '300', 'Wine', 'wine05.jpg', 5, 1),\n\t(6, 'Beer', '30', 'Kingfisher', 'beer01.jpg', 3, 1),\n\t(7, 'Beer', '20', 'Heniken', 'beer02.jpg', 10, 1),\n\t(8, 'Beer', '10', 'Fresh beer', 'beer03.jpg', 20, 1),\n\t(9, 'Beer', '5', 'Beer', 'beer04.jpg', 100, 1),\n\t(10, 'Apple', '7', 'Sweet apple', 'apple01.jpg', 200, 2),\n\t(11, 'Apple', '8', 'Delicious apple', 'apple02.jpg', 10, 2),\n\t(12, 'Apple', '4', 'Red apple', 'apple03.jpg', 500, 2),\n\t(13, 'Apple', '3', 'Blue apple', 'apple04.jpg', 20, 2),\n\t(14, 'Apple', '7', 'Big apple', 'apple05.jpg', 10, 2),\n\t(15, 'Banana', '20', 'Sweet banana', 'banana01.jpg', 20, 2),\n\t(16, 'Banana', '15', 'Delicious banana', 'banana02.jpg', 30, 2),\n\t(17, 'Banana', '25', 'Good banana', 'banana03.jpg', 10, 2),\n\t(18, 'Beef', '10', 'Meat', 'meat01.jpg', 5, 3),\n\t(19, 'Pork', '20', 'Pork', 'meat02.jpg', 10, 3),\n\t(20, 'Lamb', '20', 'Lamb', 'meat03.jpg', 20, 3),\n\t(21, 'Chicken', '15', 'Chicken', 'meat04.jpg', 15, 3),\n\t(22, 'Egg', '3', 'Egg', 'egg01.jpg', 30, 3),\n\t(23, 'Tomato', '2', 'Tomato', 'tomatos.jpg', 10, 4),\n\t(24, 'Cabbage', '5', 'Cabbage', 'cabbage01.jpg', 50, 4),\n\t(25, 'Carrot', '1', 'Carrot', 'carrot01.jpg', 20, 4);"
13331,016087,20,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: REPLACE","an SQL statement,",REPLACE,None,None,"REPLACE INTO `users` (`userId`, `email`, `password`) VALUES\n\t(1, 'Keshavtrivedi1145@gmail.com', '0cc175b9c0f1b6a831c399e269772661');"
22859,016738,50,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: REPLACE","an SQL statement,",REPLACE,None,None,"REPLACE INTO groups VALUES (-1,'',now(),now());"
22860,016738,51,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: REPLACE","an SQL statement,",REPLACE,None,None,"REPLACE INTO users VALUES (1,'root',1,'hGyVtnlrHBskPMTU5frLr59NWpU=',""<?xml version='1.0'?>\n<CAPABILITIES admin='true'>\n <Report>\n <Asr>\n <Chromocode normal='60' questionable='58' />\n </Asr>\n <Business>\n <Profit normal='5' questionable='0' />\n </Business>\n <Billing />\n </Report>\n <Alarm>\n <Log read='false' readwrite='true' />\n <CDR read='false' readwrite='true' />\n </Alarm>\n</CAPABILITIES>"",now(),now());"


#### end

In [53]:
no_replace = no_go[
    ~((no_go.found == 'REPLACE') &
    (no_go.original.str.contains('REPLACE INTO',case=False,regex=False)))
]


In [56]:
end_check = no_replace[
    (no_replace.found == 'END')
]
print(get_stmt_and_file_count(end_check))

[########################################] | 100% Completed | 9.64 ss
[########################################] | 100% Completed | 9.54 ss
(18361, 1726)


In [55]:
end_check.head()

[########################################] | 100% Completed | 235.80 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
80,014764,32,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: END","an SQL statement,",END,None,None,END DS2_TYPES;
2740,015100,57,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: END","an SQL statement,",END,None,None,"END\n$$\n\n--\n-- Создать процедуру `partInfo`\n--\nCREATE PROCEDURE partInfo (id int, shop int)\nBEGIN\n SELECT\n p.upc,\n p.name,\n p.price,\n c.id AS 'category',\n c.name AS 'category_name',\n s.id AS 'supplier',\n s.name AS 'supplier_name',\n p.description,\n p.image,\n ps.count AS 'in_stock'\n FROM part p\n JOIN category c\n ON p.id_category = c.id\n JOIN supplier s\n ON p.id_supplier = s.id\n JOIN part_shop ps\n ON p.id = ps.id_part\n JOIN shop s1\n ON ps.id_shop = s1.id\n WHERE p.id = id\n AND (s1.id = shop\n OR shop = 0);\nEND\n$$\n\nDELIMITER ;"
3431,015155,133,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: END","an SQL statement,",END,None,None,END DECISION_ARTICULO;
3441,015155,143,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: END","an SQL statement,",END,None,None,END LISTADO_COSTES;
3455,015155,157,"Query parsing failed.\n\tsql parser error: Expected an SQL statement, found: END","an SQL statement,",END,None,None,END EXPERTO_MODERADOR;


#### no END, final

In [57]:
no_end = no_replace[
    ~(no_replace.found == 'END')
]

In [163]:
print(get_stmt_and_file_count(no_end))

[########################################] | 100% Completed | 9.54 ss
[########################################] | 100% Completed | 9.80 ss
(282113, 28814)


In [285]:
to_write_down_hard.append(no_end.compute(scheduler='processes'))

[########################################] | 100% Completed | 8.96 ss


### Expected end of statement

In [60]:
expected_end_of_statement = expected_only[expected_only.expected == 'end of statement,']
expected_end_of_statement['found'].value_counts().compute(scheduler='processes')

[########################################] | 100% Completed | 7.75 ss


AUTO_INCREMENT       415328
,                    303977
COMMENT              85031 
DEFAULT              45903 
INCREMENT            40312 
                     ...   
Player_Struture      1     
Price                1     
Principais           1     
Products             1     
`CARR_SHBX_PARTY`    1     
Name: found, Length: 4880, dtype: int64

In [61]:
print(get_stmt_and_file_count(expected_end_of_statement))

[########################################] | 100% Completed | 6.74 ss
[########################################] | 100% Completed | 8.37 ss
(1257021, 152477)


In [105]:
no_rowformat['found'].value_counts().compute(scheduler='processes')

[########################################] | 100% Completed | 9.04 ss


AFTER                15479
ENABLE               14337
CONSTRAINTS          13442
CHARACTER            12755
constraints          12035
                     ...  
Player_Struture      1    
Price                1    
Principais           1    
Products             1    
`CARR_SHBX_PARTY`    1    
Name: found, Length: 4879, dtype: int64

#### AUTO_INCREMENT

In [64]:
autoincr_check = expected_end_of_statement[
    (expected_end_of_statement.found == 'AUTO_INCREMENT') &
    (expected_end_of_statement.original.str.contains('ENGINE',case=False,regex=False))
]
print(get_stmt_and_file_count(autoincr_check))

[########################################] | 100% Completed | 8.28 ss
[########################################] | 100% Completed | 10.79 s
(414166, 56351)


In [65]:
autoincr_check[autoincr_check.file_id != '014748'].head()

[########################################] | 100% Completed | 221.82 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
22,014752,0,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: AUTO_INCREMENT","end of statement,",AUTO_INCREMENT,None,None,"CREATE TABLE `cart` (\n `Id` int NOT NULL AUTO_INCREMENT,\n `TotalQty` int NOT NULL,\n PRIMARY KEY (`Id`)\n) ENGINE=InnoDB AUTO_INCREMENT=5 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;"
23,014752,1,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: AUTO_INCREMENT","end of statement,",AUTO_INCREMENT,None,None,"CREATE TABLE `customer` (\n `Id` int NOT NULL AUTO_INCREMENT,\n `age` int NOT NULL,\n `phoneNo` varchar(255) DEFAULT NULL,\n `username` varchar(255) DEFAULT NULL,\n `password` varchar(255) DEFAULT NULL,\n `street` varchar(50) DEFAULT NULL,\n `country` varchar(50) DEFAULT NULL,\n `firstName` varchar(50) DEFAULT NULL,\n `middleName` varchar(50) DEFAULT NULL,\n `lastName` varchar(50) DEFAULT NULL,\n `city` varchar(50) DEFAULT NULL,\n PRIMARY KEY (`Id`)\n) ENGINE=InnoDB AUTO_INCREMENT=2 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;"
24,014752,2,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: AUTO_INCREMENT","end of statement,",AUTO_INCREMENT,None,None,"CREATE TABLE `item` (\n `Id` int NOT NULL AUTO_INCREMENT,\n `Name` varchar(255) DEFAULT NULL,\n `Qty` int NOT NULL,\n `price` int NOT NULL,\n PRIMARY KEY (`Id`)\n) ENGINE=InnoDB AUTO_INCREMENT=8 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;"
25,014752,4,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: AUTO_INCREMENT","end of statement,",AUTO_INCREMENT,None,None,"CREATE TABLE `payment` (\n `Id` int NOT NULL AUTO_INCREMENT,\n `CartId` int NOT NULL,\n `Price` bigint NOT NULL,\n PRIMARY KEY (`Id`),\n KEY `FKpayment613095` (`CartId`),\n CONSTRAINT `FKpayment613095` FOREIGN KEY (`CartId`) REFERENCES `cart` (`Id`)\n) ENGINE=InnoDB AUTO_INCREMENT=5 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;"
26,014752,5,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: AUTO_INCREMENT","end of statement,",AUTO_INCREMENT,None,None,"CREATE TABLE `order` (\n `Id` int NOT NULL AUTO_INCREMENT,\n `CartId` int NOT NULL,\n `PaymentId` int NOT NULL,\n `CustomerId` int NOT NULL,\n `Date` date DEFAULT NULL,\n `TotalPrice` bigint NOT NULL,\n PRIMARY KEY (`Id`),\n KEY `FKorder465284` (`CustomerId`),\n KEY `FKorder777220` (`PaymentId`),\n KEY `FKorder301676` (`CartId`),\n CONSTRAINT `FKorder301676` FOREIGN KEY (`CartId`) REFERENCES `cart` (`Id`),\n CONSTRAINT `FKorder465284` FOREIGN KEY (`CustomerId`) REFERENCES `customer` (`Id`),\n CONSTRAINT `FKorder777220` FOREIGN KEY (`PaymentId`) REFERENCES `payment` (`Id`)\n) ENGINE=InnoDB AUTO_INCREMENT=4 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;"


#### comma

In [66]:
no_autoincr = expected_end_of_statement[
    ~((expected_end_of_statement.found == 'AUTO_INCREMENT') &
    (expected_end_of_statement.original.str.contains('ENGINE',case=False,regex=False)))
]

In [72]:
comma_check = no_autoincr[
    (no_autoincr.found == ',') &
    (no_autoincr.original.str.contains('ALTER TABLE',case=False,regex=False))
]
print(get_stmt_and_file_count(comma_check))

[########################################] | 100% Completed | 9.33 ss
[########################################] | 100% Completed | 8.89 ss
(301593, 35689)


In [74]:
double_add_check = comma_check

double_add_check['add_chk'] = double_add_check.apply(
    lambda row:
    1 if row['original'].split().count('ADD') > 1 else 0,
    axis=1,
    meta=('add_chk', 'int')
)

double_add_check[double_add_check.add_chk == 1]['file_id'].count().compute(scheduler='processes')

[########################################] | 100% Completed | 12.58 s


271564

In [76]:
#comma_check.head()
comma_check[comma_check.add_chk == 0].head()

[########################################] | 100% Completed | 265.80 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original,add_chk
786,014868,63,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: ,","end of statement,",",",None,None,"ALTER TABLE `civicrm_saved_search` DROP `query`,\n\t\t\t\t\t DROP `domain_id`,\t\n\t\t\t\t\t DROP `is_active`;",0
2796,015102,29,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: ,","end of statement,",",",None,None,"alter table account \n\tadd constraint fk_prnt_acnt foreign key (parent_aid) references account (id) on delete set null,\t\n\tadd constraint fk_a_pi foreign key (pi_id) references payment_info (id) on delete set null,\n\tadd constraint fk_a_c foreign key (cur_id) references currency (id) on delete set null;",0
2797,015102,31,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: ,","end of statement,",",",None,None,"alter table customer_account \n\tadd constraint fk_ca_s foreign key (visitor_id) references visitor (id) on delete set null,\n\tadd constraint fk_ca_c foreign key (customer_id) references account (id) on delete cascade,\n\tadd constraint fk_ca_a foreign key (aid) references account (id) on delete cascade;",0
2798,015102,32,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: ,","end of statement,",",",None,None,"alter table account_history \n\tadd constraint fk_ah_pt foreign key (pt_id) references payment_trans (id) on delete set null,\n\tadd constraint fk_ah_a foreign key (aid) references account (id) on delete cascade;",0
2799,015102,33,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: ,","end of statement,",",",None,None,"alter table account_address \n\tadd constraint fk_aadr_adr foreign key (address_id) references address (id) on delete cascade,\n\tadd constraint fk_aadr_a foreign key (aid) references account (id) on delete cascade;",0


In [71]:
comma_check['original'].value_counts().compute(scheduler='processes')

[########################################] | 100% Completed | 10.89 s


--\n-- Indexes for table `users`\n--\nALTER TABLE `users`\n  ADD PRIMARY KEY (`id`),\n  ADD UNIQUE KEY `users_email_unique` (`email`);                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     1663
--\n-- Indexes for table `failed_jobs`\n--\nALTER TABLE `failed_jobs`\n  ADD PRIMARY KEY (`id`),\n  ADD UNIQUE KEY `failed_jobs_uuid_unique` (`uuid`);                                                                  

#### COMMENT

In [78]:
no_comma = no_autoincr[
    ~((no_autoincr.found == ',') &
    (no_autoincr.original.str.contains('ALTER TABLE',case=False,regex=False)))
]

In [82]:
comment_check = no_comma[
    (no_comma.found == 'COMMENT') &
    (no_comma.original.str.contains('ENGINE',case=False,regex=False))
]
print(get_stmt_and_file_count(comment_check))

[########################################] | 100% Completed | 10.49 s
[########################################] | 100% Completed | 11.59 s
(77967, 10498)


In [86]:
comment_check_2 = no_comma[
    (no_comma.found == 'COMMENT') &
    (~no_comma.original.str.contains('ENGINE',case=False,regex=False))
]
print(get_stmt_and_file_count(comment_check_2))

[########################################] | 100% Completed | 7.91 ss
[########################################] | 100% Completed | 8.72 ss
(7064, 881)


In [85]:
comment_check_2.head()

[########################################] | 100% Completed | 247.67 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
19070,016480,3,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: COMMENT","end of statement,",COMMENT,None,None,"-- 직책\nCREATE TABLE `native_jdbc`.`Title` (\n\t`tno` INT NOT NULL COMMENT '직책코드', -- 직책코드\n\t`tname` VARCHAR(20) NOT NULL COMMENT '직책명' -- 직책명\n)\nCOMMENT '직책';"
19071,016480,5,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: COMMENT","end of statement,",COMMENT,None,None,"-- 부서\nCREATE TABLE `native_jdbc`.`Department` (\n\t`deptno` INT NOT NULL COMMENT '부서번호', -- 부서번호\n\t`deptname` VARCHAR(20) NOT NULL COMMENT '부서명', -- 부서명\n\t`floor` INT NULL COMMENT '위치' -- 위치\n)\nCOMMENT '부서';"
19072,016480,7,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: COMMENT","end of statement,",COMMENT,None,None,"-- 사원\nCREATE TABLE `native_jdbc`.`employee` (\n\t`empno` INT NOT NULL COMMENT '사원번호', -- 사원번호\n\t`empname` VARCHAR(20) NULL COMMENT '사원명', -- 사원명\n\t`title` INT NULL COMMENT '직책', -- 직책\n\t`manager` INT NULL COMMENT '직속상사', -- 직속상사\n\t`salary` INT NULL COMMENT '급여', -- 급여\n\t`dept` INT NULL COMMENT '부서', -- 부서\n\t`deptno` INT NULL COMMENT '부서번호' -- 부서번호\n)\nCOMMENT '사원';"
25455,017027,71,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: COMMENT","end of statement,",COMMENT,None,None,"-- --------------------------------------------------------\nCREATE TABLE `mutation_event` (\n `MUTATION_EVENT_ID` int(255) NOT NULL auto_increment,\n `ENTREZ_GENE_ID` int(11) NOT NULL,\n `CHR` varchar(5),\n `START_POSITION` bigint(20),\n `END_POSITION` bigint(20),\n `REFERENCE_ALLELE` varchar(255),\n `TUMOR_SEQ_ALLELE` varchar(255),\n `PROTEIN_CHANGE` varchar(255),\n `MUTATION_TYPE` varchar(255) COMMENT 'e.g. Missense, Nonsence, etc.',\n `FUNCTIONAL_IMPACT_SCORE` varchar(50) COMMENT 'Result from OMA/XVAR.',\n `FIS_VALUE` float,\n `LINK_XVAR` varchar(500) COMMENT 'Link to OMA/XVAR Landing Page for the specific mutation.',\n `LINK_PDB` varchar(500),\n `LINK_MSA` varchar(500),\n `NCBI_BUILD` varchar(10),\n `STRAND` varchar(2),\n `VARIANT_TYPE` varchar(15),\n `DB_SNP_RS` varchar(25),\n `DB_SNP_VAL_STATUS` varchar(255),\n `ONCOTATOR_DBSNP_RS` varchar(255),\n `ONCOTATOR_REFSEQ_MRNA_ID` varchar(64),\n `ONCOTATOR_CODON_CHANGE` varchar(255),\n `ONCOTATOR_UNIPROT_ENTRY_NAME` varchar(64),\n `ONCOTATOR_UNIPROT_ACCESSION` varchar(64),\n `ONCOTATOR_PROTEIN_POS_START` int(11),\n `ONCOTATOR_PROTEIN_POS_END` int(11),\n `CANONICAL_TRANSCRIPT` boolean,\n `KEYWORD` varchar(50) DEFAULT NULL COMMENT 'e.g. truncating, V200 Missense, E338del, ',\n KEY (`KEYWORD`),\n PRIMARY KEY (`MUTATION_EVENT_ID`),\n UNIQUE (`CHR`, `START_POSITION`, `END_POSITION`, `TUMOR_SEQ_ALLELE`, `ENTREZ_GENE_ID`, `PROTEIN_CHANGE`, `MUTATION_TYPE`),\n FOREIGN KEY (`ENTREZ_GENE_ID`) REFERENCES `gene` (`ENTREZ_GENE_ID`)\n) COMMENT='Mutation Data';"
25457,017027,85,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: COMMENT","end of statement,",COMMENT,None,None,"-- --------------------------------------------------------\nCREATE TABLE `sanger_cancer_census` (\n `ENTREZ_GENE_ID` int(11) NOT NULL,\n `CANCER_SOMATIC_MUT` tinyint(1) NOT NULL,\n `CANCER_GERMLINE_MUT` tinyint(1) NOT NULL,\n `TUMOR_TYPES_SOMATIC_MUT` text NOT NULL,\n `TUMOR_TYPES_GERMLINE_MUT` text NOT NULL,\n `CANCER_SYNDROME` text NOT NULL,\n `TISSUE_TYPE` text NOT NULL,\n `MUTATION_TYPE` text NOT NULL,\n `TRANSLOCATION_PARTNER` text NOT NULL,\n `OTHER_GERMLINE_MUT` tinyint(1) NOT NULL,\n `OTHER_DISEASE` text NOT NULL,\n FOREIGN KEY (`ENTREZ_GENE_ID`) REFERENCES `gene` (`ENTREZ_GENE_ID`)\n) COMMENT='Sanger Cancer Gene Census';"


In [83]:
comment_check[comment_check.file_id != '014771'].head()

[########################################] | 100% Completed | 282.70 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
996,014904,55,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: COMMENT","end of statement,",COMMENT,None,None,"CREATE TABLE `schedule_job` (\n `job_id` bigint(20) NOT NULL AUTO_INCREMENT COMMENT '任务id',\n `bean_name` varchar(200) DEFAULT NULL COMMENT 'spring bean名称',\n `method_name` varchar(100) DEFAULT NULL COMMENT '方法名',\n `params` varchar(2000) DEFAULT NULL COMMENT '参数',\n `cron_expression` varchar(100) DEFAULT NULL COMMENT 'cron表达式',\n `status` tinyint(4) DEFAULT NULL COMMENT '任务状态',\n `remark` varchar(255) DEFAULT NULL COMMENT '备注',\n `create_time` datetime DEFAULT NULL COMMENT '创建时间',\n PRIMARY KEY (`job_id`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8 COMMENT='定时任务';"
1000,014904,62,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: COMMENT","end of statement,",COMMENT,None,None,"CREATE TABLE `schedule_job_log` (\n `log_id` bigint(20) NOT NULL AUTO_INCREMENT COMMENT '任务日志id',\n `job_id` bigint(20) NOT NULL COMMENT '任务id',\n `bean_name` varchar(200) DEFAULT NULL COMMENT 'spring bean名称',\n `method_name` varchar(100) DEFAULT NULL COMMENT '方法名',\n `params` varchar(2000) DEFAULT NULL COMMENT '参数',\n `status` tinyint(4) NOT NULL COMMENT '任务状态 0：成功 1：失败',\n `error` varchar(2000) DEFAULT NULL COMMENT '失败信息',\n `times` int(11) NOT NULL COMMENT '耗时(单位：毫秒)',\n `create_time` datetime DEFAULT NULL COMMENT '创建时间',\n PRIMARY KEY (`log_id`),\n KEY `job_id` (`job_id`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8 COMMENT='定时任务日志';"
1008,014904,76,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: COMMENT","end of statement,",COMMENT,None,None,"CREATE TABLE `sys_log` (\n `id` bigint(20) NOT NULL AUTO_INCREMENT,\n `username` varchar(50) DEFAULT NULL COMMENT '用户名',\n `operation` varchar(50) DEFAULT NULL COMMENT '用户操作',\n `method` varchar(200) DEFAULT NULL COMMENT '请求方法',\n `params` varchar(5000) DEFAULT NULL COMMENT '请求参数',\n `ip` varchar(64) DEFAULT NULL COMMENT 'IP地址',\n `create_date` datetime DEFAULT NULL COMMENT '创建时间',\n PRIMARY KEY (`id`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8 COMMENT='系统日志';"
1012,014904,83,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: COMMENT","end of statement,",COMMENT,None,None,"CREATE TABLE `sys_menu` (\n `menu_id` bigint(20) NOT NULL AUTO_INCREMENT,\n `parent_id` bigint(20) DEFAULT NULL COMMENT '父菜单ID，一级菜单为0',\n `name` varchar(50) DEFAULT NULL COMMENT '菜单名称',\n `url` varchar(200) DEFAULT NULL COMMENT '菜单URL',\n `perms` varchar(500) DEFAULT NULL COMMENT '授权(多个用逗号分隔，如：user:list,user:create)',\n `type` int(11) DEFAULT NULL COMMENT '类型 0：目录 1：菜单 2：按钮',\n `icon` varchar(50) DEFAULT NULL COMMENT '菜单图标',\n `order_num` int(11) DEFAULT NULL COMMENT '排序',\n PRIMARY KEY (`menu_id`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8 COMMENT='菜单管理';"
1016,014904,90,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: COMMENT","end of statement,",COMMENT,None,None,"CREATE TABLE `sys_role` (\n `role_id` bigint(20) NOT NULL AUTO_INCREMENT,\n `role_name` varchar(100) DEFAULT NULL COMMENT '角色名称',\n `remark` varchar(100) DEFAULT NULL COMMENT '备注',\n `create_user_id` bigint(20) DEFAULT NULL COMMENT '创建者ID',\n `create_time` datetime DEFAULT NULL COMMENT '创建时间',\n PRIMARY KEY (`role_id`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8 COMMENT='角色';"


#### DEFAULT

In [87]:
no_comment = no_comma[
    ~(no_comma.found == 'COMMENT')
]

In [90]:
default_check = no_comment[
    (no_comment.found == 'DEFAULT') &
    (no_comment.original.str.contains('CHARACTER SET',regex=False,case=False))
]
print(get_stmt_and_file_count(default_check))

[########################################] | 100% Completed | 8.87 ss
[########################################] | 100% Completed | 8.96 ss
(42590, 13300)


In [89]:
default_check.head()

[########################################] | 100% Completed | 254.15 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
186,014796,3,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: DEFAULT","end of statement,",DEFAULT,None,None,-- -----------------------------------------------------\n-- Schema mydb\n-- -----------------------------------------------------\n-- -----------------------------------------------------\n-- Schema db_Srv_Source\n-- -----------------------------------------------------\n\n-- -----------------------------------------------------\n-- Schema db_Srv_Source\n-- -----------------------------------------------------\nCREATE SCHEMA IF NOT EXISTS `db_Srv_Source` DEFAULT CHARACTER SET utf8 ;
370,014834,7,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: DEFAULT","end of statement,",DEFAULT,None,None,--\n-- Database: `epiz_28109172_ukusisrema`\n--\nCREATE DATABASE IF NOT EXISTS `epiz_28109172_ukusisrema` DEFAULT CHARACTER SET utf8 COLLATE utf8_unicode_ci;
771,014868,4,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: DEFAULT","end of statement,",DEFAULT,None,None,"-- /*******************************************************\n-- *\n-- * civicrm_membership_log\n-- *\n-- * Logs actions which affect a Membership record (signup, status override, renewal, etc.)\n-- *\n-- *******************************************************/\nCREATE TABLE civicrm_membership_log (\n\n\n id int unsigned NOT NULL AUTO_INCREMENT ,\n membership_id int unsigned NOT NULL COMMENT 'FK to Membership table',\n status_id int unsigned NOT NULL COMMENT 'New status assigned to membership by this action. FK to Membership Status',\n start_date date COMMENT 'New membership period start date',\n end_date date COMMENT 'New membership period expiration date.',\n modified_id int unsigned COMMENT 'FK to Contact ID of person under whose credentials this data modification was made.',\n modified_date date COMMENT 'Date this membership modification action was logged.' \n,\n PRIMARY KEY ( id )\n \n \n, \n FOREIGN KEY (membership_id) REFERENCES civicrm_membership(id)\n, \n FOREIGN KEY (status_id) REFERENCES civicrm_membership_status(id)\n, \n FOREIGN KEY (modified_id) REFERENCES civicrm_contact(id)\n \n) ENGINE=InnoDB DEFAULT CHARACTER SET utf8 COLLATE utf8_unicode_ci ;"
772,014868,5,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: DEFAULT","end of statement,",DEFAULT,None,None,"-- /*******************************************************\n-- *\n-- * civicrm_membership_payment\n-- *\n-- * Membership Payment\n-- *\n-- *******************************************************/\nCREATE TABLE civicrm_membership_payment (\n\n\n id int unsigned NOT NULL AUTO_INCREMENT ,\n membership_id int unsigned NOT NULL COMMENT 'FK to Membership table',\n payment_entity_table varchar(128) COMMENT 'Table being referenced for payment entity.',\n payment_entity_id int unsigned COMMENT 'FK to table with payment record (expected usage is civicrm_contribution).' \n,\n PRIMARY KEY ( id )\n \n , INDEX index_payment_entity(\n payment_entity_table\n , payment_entity_id\n )\n \n, \n FOREIGN KEY (membership_id) REFERENCES civicrm_membership(id)\n \n) ENGINE=InnoDB DEFAULT CHARACTER SET utf8 COLLATE utf8_unicode_ci ;"
1478,014946,143,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: DEFAULT","end of statement,",DEFAULT,None,None,"-- /*******************************************************\n-- *\n-- * Create new tables\n-- *\n-- *******************************************************/\n\n-- /*******************************************************\n-- *\n-- * civicrm_address_format\n-- *\n-- *******************************************************/\nCREATE TABLE `civicrm_address_format` (\n\n\n `id` int unsigned NOT NULL AUTO_INCREMENT COMMENT 'Address Format Id',\n `format` text COMMENT 'The format of an address' \n,\n PRIMARY KEY ( `id` )\n \n \n \n) ENGINE=InnoDB DEFAULT CHARACTER

#### INCREMENT

In [91]:
no_default = no_comment[
    ~((no_comment.found == 'DEFAULT') &
    (no_comment.original.str.contains('CHARACTER SET',regex=False,case=False)))
]

In [94]:
increment_check = no_default[
    (no_default.found == 'INCREMENT') &
    (no_comment.original.str.contains('CREATE SEQUENCE',regex=False,case=False))
]
print(get_stmt_and_file_count(increment_check))

[########################################] | 100% Completed | 7.89 ss
[########################################] | 100% Completed | 10.43 s
(40297, 5210)


In [93]:
increment_check.head()

[########################################] | 100% Completed | 240.19 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
292,014818,11,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: INCREMENT","end of statement,",INCREMENT,None,None,--\n-- Name: smtp_transaction_recipients_smtp_transaction_recipient_id_seq; Type: SEQUENCE; Schema: public; Owner: mipanel\n--\n\nCREATE SEQUENCE smtp_transaction_recipients_smtp_transaction_recipient_id_seq\n START WITH 1\n INCREMENT BY 1\n NO MAXVALUE\n NO MINVALUE\n CACHE 1;
296,014818,16,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: INCREMENT","end of statement,",INCREMENT,None,None,--\n-- Name: smtp_transactions_smtp_transaction_id_seq; Type: SEQUENCE; Schema: public; Owner: mipanel\n--\n\nCREATE SEQUENCE smtp_transactions_smtp_transaction_id_seq\n START WITH 1\n INCREMENT BY 1\n NO MAXVALUE\n NO MINVALUE\n CACHE 1;
845,014875,50,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: INCREMENT","end of statement,",INCREMENT,None,None,--\n-- Name: processing_studies_processing_studies_id_seq; Type: SEQUENCE; Schema: public; Owner: seqware\n--\n\nCREATE SEQUENCE processing_studies_processing_studies_id_seq\n START WITH 1\n INCREMENT BY 1\n NO MAXVALUE\n NO MINVALUE\n CACHE 1;
849,014875,61,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: INCREMENT","end of statement,",INCREMENT,None,None,--\n-- Name: file_type_file_type_id_seq; Type: SEQUENCE; Schema: public; Owner: seqware\n--\n\nCREATE SEQUENCE file_type_file_type_id_seq\n START WITH 1\n INCREMENT BY 1\n NO MAXVALUE\n NO MINVALUE\n CACHE 1;
853,014875,72,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: INCREMENT","end of statement,",INCREMENT,None,None,--\n-- Name: processing_experiments_processing_experiments_id_seq; Type: SEQUENCE; Schema: public; Owner: seqware\n--\n\nCREATE SEQUENCE processing_experiments_processing_experiments_id_seq\n START WITH 1\n INCREMENT BY 1\n NO MAXVALUE\n NO MINVALUE\n CACHE 1;


#### DEFERRABLE

In [95]:
no_increment = no_default[
    ~((no_default.found == 'INCREMENT') &
    (no_comment.original.str.contains('CREATE SEQUENCE',regex=False,case=False)))
]

In [99]:
deff_check = no_increment[
    ((no_increment.found == 'DEFERRABLE') &
    (no_increment.original.str.contains('CONSTRAINT',regex=False,case=False)))
]
print(get_stmt_and_file_count(deff_check))

[########################################] | 100% Completed | 9.30 ss
[########################################] | 100% Completed | 9.84 ss
(29894, 997)


In [98]:
deff_check.head()

[########################################] | 100% Completed | 235.23 ms


c:\Users\geacu\anaconda3\envs\windows\lib\site-packages\dask\core.py:119: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
8173,015546,528,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: DEFERRABLE","end of statement,",DEFERRABLE,None,None,--\n-- Name: auth_group_permissio_permission_id_84c5c92e_fk_auth_perm; Type: FK CONSTRAINT; Schema: public; Owner: iftttuser\n--\n\nALTER TABLE ONLY public.auth_group_permissions\n ADD CONSTRAINT auth_group_permissio_permission_id_84c5c92e_fk_auth_perm FOREIGN KEY (permission_id) REFERENCES public.auth_permission(id) DEFERRABLE INITIALLY DEFERRED;
8174,015546,529,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: DEFERRABLE","end of statement,",DEFERRABLE,None,None,--\n-- Name: auth_group_permissions_group_id_b120cbf9_fk_auth_group_id; Type: FK CONSTRAINT; Schema: public; Owner: iftttuser\n--\n\nALTER TABLE ONLY public.auth_group_permissions\n ADD CONSTRAINT auth_group_permissions_group_id_b120cbf9_fk_auth_group_id FOREIGN KEY (group_id) REFERENCES public.auth_group(id) DEFERRABLE INITIALLY DEFERRED;
8175,015546,530,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: DEFERRABLE","end of statement,",DEFERRABLE,None,None,--\n-- Name: auth_permission_content_type_id_2f476e4b_fk_django_co; Type: FK CONSTRAINT; Schema: public; Owner: iftttuser\n--\n\nALTER TABLE ONLY public.auth_permission\n ADD CONSTRAINT auth_permission_content_type_id_2f476e4b_fk_django_co FOREIGN KEY (content_type_id) REFERENCES public.django_content_type(id) DEFERRABLE INITIALLY DEFERRED;
8176,015546,531,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: DEFERRABLE","end of statement,",DEFERRABLE,None,None,--\n-- Name: auth_user_groups_group_id_97559544_fk_auth_group_id; Type: FK CONSTRAINT; Schema: public; Owner: iftttuser\n--\n\nALTER TABLE ONLY public.auth_user_groups\n ADD CONSTRAINT auth_user_groups_group_id_97559544_fk_auth_group_id FOREIGN KEY (group_id) REFERENCES public.auth_group(id) DEFERRABLE INITIALLY DEFERRED;
8177,015546,532,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: DEFERRABLE","end of statement,",DEFERRABLE,None,None,--\n-- Name: auth_user_groups_user_id_6a12ed8b_fk_auth_user_id; Type: FK CONSTRAINT; Schema: public; Owner: iftttuser\n--\n\nALTER TABLE ONLY public.auth_user_groups\n ADD CONSTRAINT auth_user_groups_user_id_6a12ed8b_fk_auth_user_id FOREIGN KEY (user_id) REFERENCES public.auth_user(id) DEFERRABLE INITIALLY DEFERRED;


#### ROW_FORMAT

In [100]:
no_deff = no_increment[
    ~((no_increment.found == 'DEFERRABLE') &
    (no_increment.original.str.contains('CONSTRAINT',regex=False,case=False)))
]

In [103]:
rowformat_check = no_deff[
    ((no_deff.found == 'ROW_FORMAT') & 
    (no_deff.original.str.contains('ENGINE',regex=False,case=False)))
]
print(get_stmt_and_file_count(rowformat_check))

[########################################] | 100% Completed | 8.27 ss
[########################################] | 100% Completed | 10.04 s
(21572, 3167)


In [102]:
rowformat_check.head()

[########################################] | 100% Completed | 283.39 ms


c:\Users\geacu\anaconda3\envs\windows\lib\site-packages\dask\core.py:119: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
107,014771,117,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: ROW_FORMAT","end of statement,",ROW_FORMAT,None,None,"CREATE TABLE `dp_admin_log` (\n `id` int(11) unsigned NOT NULL AUTO_INCREMENT COMMENT '主键',\n `action_id` int(11) unsigned NOT NULL DEFAULT '0' COMMENT '行为id',\n `user_id` int(11) unsigned NOT NULL DEFAULT '0' COMMENT '执行用户id',\n `action_ip` bigint(20) NOT NULL COMMENT '执行行为者ip',\n `model` varchar(50) NOT NULL DEFAULT '' COMMENT '触发行为的表',\n `record_id` int(11) unsigned NOT NULL DEFAULT '0' COMMENT '触发行为的数据id',\n `remark` longtext NOT NULL COMMENT '日志备注',\n `status` tinyint(2) NOT NULL DEFAULT '1' COMMENT '状态',\n `create_time` int(11) unsigned NOT NULL DEFAULT '0' COMMENT '执行行为的时间',\n PRIMARY KEY (`id`),\n KEY `action_ip_ix` (`action_ip`),\n KEY `action_id_ix` (`action_id`),\n KEY `user_id_ix` (`user_id`)\n) ENGINE=MyISAM DEFAULT CHARSET=utf8 ROW_FORMAT=FIXED COMMENT='行为日志表';"
471,014839,21,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: ROW_FORMAT","end of statement,",ROW_FORMAT,None,None,"-- --------------------------------------------------------\n\n--\n-- 테이블 구조 `psu_order`\n--\n\nCREATE TABLE IF NOT EXISTS `psu_order` (\n `order_id` int(12) unsigned NOT NULL,\n `item_id` int(12) unsigned NOT NULL,\n `user_id` varchar(255) NOT NULL DEFAULT 'nonmember',\n `order_price` int(12) unsigned NOT NULL DEFAULT 0,\n `order_code` varchar(255) NOT NULL,\n `order_pwd` varchar(255) NOT NULL,\n `order_tracking` varchar(100) NOT NULL,\n `order_tracking_num` varchar(255) NOT NULL,\n `order_register` datetime NOT NULL DEFAULT '0000-00-00 00:00:00'\n) ENGINE=InnoDB DEFAULT CHARSET=utf8 ROW_FORMAT=COMPACT;"
472,014839,22,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: ROW_FORMAT","end of statement,",ROW_FORMAT,None,None,"-- --------------------------------------------------------\n\n--\n-- 테이블 구조 `psu_order_user`\n--\n\nCREATE TABLE IF NOT EXISTS `psu_order_user` (\n `order_user_id` int(12) unsigned NOT NULL,\n `order_id` int(12) unsigned NOT NULL,\n `order_user_name` varchar(50) NOT NULL,\n `order_user_phone` varchar(50) NOT NULL,\n `order_user_email` varchar(100) NOT NULL,\n `order_user_country` varchar(255) NOT NULL,\n `order_user_city` varchar(255) NOT NULL,\n `order_user_state` varchar(255) NOT NULL,\n `order_user_zip` varchar(100) NOT NULL,\n `order_user_addr` varchar(255) NOT NULL,\n `order_user_re_name` varchar(50) NOT NULL,\n `order_user_re_phone` varchar(50) NOT NULL,\n `order_user_re_email` varchar(100) NOT NULL,\n `order_user_re_country` varchar(255) NOT NULL,\n `order_user_re_city` varchar(255) NOT NULL,\n `order_user_re_state` varchar(255) NOT NULL,\n `order_user_re_zip` varchar(100) NOT NULL,\n `order_user_re_addr` varchar(255) NOT NULL,\n `order_user_register` datetime NOT NULL DEFAULT '0000-00-00 00:00:00'\n) ENGINE=InnoDB DEFAULT CHARSET=utf8 ROW_FORMAT=COMPACT;"
686,014854,885,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: ROW_FORMAT","end of statement,",ROW_FORMAT,None,None,"CREATE TABLE `fa_sms` (\n `id` int(10) unsigned NOT NULL AUTO_INCREMENT COMMENT 'ID',\n `event` varchar(30) NOT NULL DEFAULT '' COMMENT '事件',\n `mobile` varchar(20) NOT NULL DEFAULT '' COMMENT '手机号',\n `code` varchar(10) NOT NULL DEFAULT '' COMMENT '验证码',\n `times` int(10) unsigned NOT NULL DEFAULT '0' COMMENT '验证次数',\n `ip` varchar(30) NOT NULL DEFAULT '' COMMENT 'IP',\n `createtime` int(10) unsigned DEFAULT '0' COMMENT '创建时间',\n PRIMARY KEY (`id`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8 ROW_FORMAT=COMPACT COMMENT='短信验证码表';"
1787,014972,33,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: ROW_FORMAT","end of statement,",ROW_FORMAT,None,None,"CREATE TABLE `clients` (\n `id` int(11) NOT NULL AUTO_INCREMENT,\n `title` varchar(255) DEFAULT NULL,\n `link` varchar(255) DEFAULT NULL,\n `image` varchar(255) 

#### AFTER

In [104]:
no_rowformat = no_deff[
    ~((no_deff.found == 'ROW_FORMAT') & 
    (no_deff.original.str.contains('ENGINE',regex=False,case=False)))
]

In [108]:
after_check = no_rowformat[
    ((no_rowformat.found == 'AFTER') &
    (no_rowformat.original.str.contains('ALTER TABLE',regex=False,case=False)))
]
print(get_stmt_and_file_count(after_check))

[########################################] | 100% Completed | 9.19 ss
[########################################] | 100% Completed | 10.78 s
(15474, 2526)


In [107]:
after_check.head()

[########################################] | 100% Completed | 348.60 ms


c:\Users\geacu\anaconda3\envs\windows\lib\site-packages\dask\core.py:119: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
563,014846,8,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: AFTER","end of statement,",AFTER,None,None,"ALTER TABLE `hotel`.`transaction` \n\tADD COLUMN `check_in_by_name` VARCHAR(45) NULL AFTER `data_customer` , \n\tADD COLUMN `check_out_by_name` VARCHAR(45) NULL AFTER `check_in_by_name` ;"
3648,015201,22,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: AFTER","end of statement,",AFTER,None,None,"ALTER TABLE user\nADD Show_db_priv enum('N','Y') COLLATE utf8_general_ci DEFAULT 'N' NOT NULL AFTER Alter_priv,\nADD Super_priv enum('N','Y') COLLATE utf8_general_ci DEFAULT 'N' NOT NULL AFTER Show_db_priv,\nADD Create_tmp_table_priv enum('N','Y') COLLATE utf8_general_ci DEFAULT 'N' NOT NULL AFTER Super_priv,\nADD Lock_tables_priv enum('N','Y') COLLATE utf8_general_ci DEFAULT 'N' NOT NULL AFTER Create_tmp_table_priv,\nADD Execute_priv enum('N','Y') COLLATE utf8_general_ci DEFAULT 'N' NOT NULL AFTER Lock_tables_priv,\nADD Repl_slave_priv enum('N','Y') COLLATE utf8_general_ci DEFAULT 'N' NOT NULL AFTER Execute_priv,\nADD Repl_client_priv enum('N','Y') COLLATE utf8_general_ci DEFAULT 'N' NOT NULL AFTER Repl_slave_priv;"
3670,015201,46,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: AFTER","end of statement,",AFTER,None,None,ALTER TABLE slow_log\n ADD COLUMN thread_id BIGINT(21) UNSIGNED NOT NULL AFTER sql_text;
3678,015201,54,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: AFTER","end of statement,",AFTER,None,None,"ALTER TABLE host ADD Create_view_priv enum('N','Y') COLLATE utf8_general_ci DEFAULT 'N' NOT NULL AFTER Lock_tables_priv;"
3680,015201,56,"Query parsing failed.\n\tsql parser error: Expected end of statement, found: AFTER","end of statement,",AFTER,None,None,"ALTER TABLE user ADD Create_view_priv enum('N','Y') COLLATE utf8_general_ci DEFAULT 'N' NOT NULL AFTER Repl_client_priv;"


#### final

In [109]:
no_after = no_rowformat[
    ~((no_rowformat.found == 'AFTER') &
    (no_rowformat.original.str.contains('ALTER TABLE',regex=False,case=False)))
]

In [166]:
print(get_stmt_and_file_count(no_after))

[########################################] | 100% Completed | 10.62 s
[########################################] | 100% Completed | 12.36 s
(306404, 50604)


In [286]:
to_write_down_hard.append(no_after.compute(scheduler='processes'))

[########################################] | 100% Completed | 9.66 ss


### Expected after ALTER

#### Found what

In [111]:
expected_alter = expected_only[expected_only.expected == 'ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,']
expected_alter['found'].value_counts().compute(scheduler='processes')

[########################################] | 100% Completed | 8.05 ss


MODIFY                             305195
OWNER                              72619 
modify                             7078  
exists                             4492  
ENGINE                             3650  
EXISTS                             3394  
AUTO_INCREMENT                     2989  
comment                            2762  
COMMENT                            2471  
WITH                               1712  
CONVERT                            1644  
auto_increment                     1451  
ENABLE                             1002  
DISABLE                            960   
owner                              552   
TYPE                               479   
disable                            455   
CHECK                              424   
enable                             418   
VOLATILE                           293   
CLUSTER                            244   
foreign                            227   
DEFAULT                            219   
set                               

In [112]:
print(get_stmt_and_file_count(expected_alter))

[########################################] | 100% Completed | 6.94 ss
[########################################] | 100% Completed | 7.28 ss
(417369, 38600)


#### MODIFY

In [118]:
modify_check = expected_alter[
    (expected_alter.found.str.contains('MODIFY',case=False,regex=False))
]
print(get_stmt_and_file_count(modify_check))

[########################################] | 100% Completed | 9.23 ss
[########################################] | 100% Completed | 9.98 ss
(312301, 30042)


In [116]:
modify_check[modify_check.file_id != '014745'].head()

[########################################] | 100% Completed | 325.99 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
43,014757,51,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: MODIFY","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",MODIFY,None,None,"--\n-- AUTO_INCREMENT de las tablas volcadas\n--\n\n--\n-- AUTO_INCREMENT de la tabla `categorias`\n--\nALTER TABLE `categorias`\n MODIFY `id` int(10) UNSIGNED NOT NULL AUTO_INCREMENT, AUTO_INCREMENT=3;"
44,014757,52,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: MODIFY","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",MODIFY,None,None,"--\n-- AUTO_INCREMENT de la tabla `data_rows`\n--\nALTER TABLE `data_rows`\n MODIFY `id` int(10) UNSIGNED NOT NULL AUTO_INCREMENT, AUTO_INCREMENT=39;"
45,014757,53,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: MODIFY","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",MODIFY,None,None,"--\n-- AUTO_INCREMENT de la tabla `data_types`\n--\nALTER TABLE `data_types`\n MODIFY `id` int(10) UNSIGNED NOT NULL AUTO_INCREMENT, AUTO_INCREMENT=6;"
46,014757,54,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: MODIFY","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",MODIFY,None,None,--\n-- AUTO_INCREMENT de la tabla `failed_jobs`\n--\nALTER TABLE `failed_jobs`\n MODIFY `id` bigint(20) UNSIGNED NOT NULL AUTO_INCREMENT;
47,014757,55,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: MODIFY","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",MODIFY,None,None,"--\n-- AUTO_INCREMENT de la tabla `menus`\n--\nALTER TABLE `menus`\n MODIFY `id` int(10) UNSIGNED NOT NULL AUTO_INCREMENT, AUTO_INCREMENT=2;"


#### OWNER

In [119]:
no_modify = expected_alter[
   (~expected_alter.found.str.contains('MODIFY',case=False,regex=False))
]
owner_check = no_modify[
    (no_modify.found.str.contains('OWNER',case=False,regex=False))
]
print(get_stmt_and_file_count(owner_check))

[########################################] | 100% Completed | 11.59 s
[########################################] | 100% Completed | 10.59 s
(73171, 5519)


In [128]:
owner_check.head()

[########################################] | 100% Completed | 254.74 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
291,014818,10,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: OWNER","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",OWNER,None,None,ALTER TABLE public.smtp_transaction_recipients OWNER TO mipanel;
293,014818,12,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: OWNER","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",OWNER,None,None,ALTER TABLE public.smtp_transaction_recipients_smtp_transaction_recipient_id_seq OWNER TO mipanel;
295,014818,15,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: OWNER","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",OWNER,None,None,ALTER TABLE public.smtp_transactions OWNER TO mipanel;
297,014818,17,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: OWNER","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",OWNER,None,None,ALTER TABLE public.smtp_transactions_smtp_transaction_id_seq OWNER TO mipanel;
843,014875,43,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: OWNER","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",OWNER,None,None,ALTER TABLE sample_hierarchy OWNER TO seqware;


#### EXISTS

In [120]:
no_owner = no_modify[
   (~no_modify.found.str.contains('OWNER',case=False,regex=False))
]
exists_check = no_owner[
    (no_owner.found.str.contains('EXISTS',case=False,regex=False))
]
print(get_stmt_and_file_count(exists_check))

[########################################] | 100% Completed | 10.19 s
[########################################] | 100% Completed | 11.75 s
(7962, 878)


In [129]:
exists_check.head()

[########################################] | 100% Completed | 249.82 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
3579,015187,2,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: exists","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",exists,None,None,alter table if exists tb_sales add constraint FKp0nn0oixeiw2fq3jivj6vnsu foreign key (seller_id) references tb_sellers;
3580,015187,5,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: exists","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",exists,None,None,alter table if exists tb_sales add constraint FKp0nn0oixeiw2fq3jivj6vnsu foreign key (seller_id) references tb_sellers;
3581,015187,8,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: exists","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",exists,None,None,alter table if exists tb_sales add constraint FKp0nn0oixeiw2fq3jivj6vnsu foreign key (seller_id) references tb_sellers;
11008,015752,9,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: exists","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",exists,None,None,alter table if exists category_sub_categories add constraint UK_bvsvfpcbk584cpyecc213hl3m unique (sub_categories_id)\n ;
11009,015752,10,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: exists","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",exists,None,None,alter table if exists subcategory_products add constraint UK_n6bw6ka1l8g1w2da33xh7oosf unique (products_id)\n ;


#### ENGINE

In [121]:
no_exists = no_owner[
   (~no_owner.found.str.contains('EXISTS',case=False,regex=False))
]
engine_check = no_owner[
    (no_owner.found.str.contains('ENGINE',case=False,regex=False))
]
print(get_stmt_and_file_count(engine_check))

[########################################] | 100% Completed | 9.85 ss
[########################################] | 100% Completed | 10.99 s
(3792, 243)


In [130]:
engine_check.head()

[########################################] | 100% Completed | 241.57 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
3663,015201,38,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: ENGINE","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",ENGINE,None,None,"ALTER TABLE func\n ENGINE=InnoDB, CONVERT TO CHARACTER SET utf8 COLLATE utf8_bin;"
3794,015201,200,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: ENGINE","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",ENGINE,None,None,ALTER TABLE servers ENGINE=InnoDB;
3795,015201,201,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: ENGINE","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",ENGINE,None,None,ALTER TABLE help_topic ENGINE=InnoDB;
3796,015201,202,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: ENGINE","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",ENGINE,None,None,ALTER TABLE help_category ENGINE=InnoDB;
3797,015201,203,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: ENGINE","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",ENGINE,None,None,ALTER TABLE help_relation ENGINE=InnoDB;


#### COMMENT

In [122]:
no_engine = no_exists[
   (~no_exists.found.str.contains('ENGINE',case=False,regex=False))
]
comment_check = no_engine[
    (no_engine.found.str.contains('COMMENT',case=False,regex=False))
]
print(get_stmt_and_file_count(comment_check))

[########################################] | 100% Completed | 10.46 s
[########################################] | 100% Completed | 9.78 ss
(5235, 465)


In [131]:
comment_check.head()

[########################################] | 100% Completed | 347.06 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
1874,014976,11,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: comment","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",comment,None,None,alter table TBL_USER comment '”√ªß±Ì';
3653,015201,28,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: comment","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",comment,None,None,alter table db comment='Database privileges';
3654,015201,29,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: comment","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",comment,None,None,alter table host comment='Host privileges; Merged with database privileges';
3655,015201,30,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: comment","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",comment,None,None,alter table user comment='Users and global privileges';
3656,015201,31,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: comment","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",comment,None,None,alter table func comment='User defined functions';


#### AUTO_INCREMENT

In [123]:
no_comment_alter = no_engine[
   (~no_engine.found.str.contains('COMMENT',case=False,regex=False))
]
auto_incr_check = no_comment_alter[
    (no_comment_alter.found.str.contains('AUTO_INCREMENT',case=False,regex=False))
]
print(get_stmt_and_file_count(auto_incr_check))

[########################################] | 100% Completed | 8.05 ss
[########################################] | 100% Completed | 6.97 ss
(4442, 881)


In [132]:
auto_incr_check.head()

[########################################] | 100% Completed | 250.12 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
6891,015397,1,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: AUTO_INCREMENT","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",AUTO_INCREMENT,None,None,ALTER TABLE bank_schema.client AUTO_INCREMENT=10000;
6893,015397,3,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: AUTO_INCREMENT","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",AUTO_INCREMENT,None,None,ALTER TABLE bank_schema.login_account AUTO_INCREMENT=60000;
6895,015397,5,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: AUTO_INCREMENT","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",AUTO_INCREMENT,None,None,ALTER TABLE bank_schema.employee AUTO_INCREMENT=20000;
6897,015397,7,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: AUTO_INCREMENT","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",AUTO_INCREMENT,None,None,ALTER TABLE bank_schema.card AUTO_INCREMENT=40000;
6899,015397,9,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: AUTO_INCREMENT","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",AUTO_INCREMENT,None,None,ALTER TABLE bank_schema.bank_account AUTO_INCREMENT=500000;


#### CONVERT

In [124]:
nu_auto_incr_alter = no_comment_alter[
   (~no_comment_alter.found.str.contains('AUTO_INCREMENT',case=False,regex=False))
]
convert_check = nu_auto_incr_alter[
    (nu_auto_incr_alter.found.str.contains('CONVERT',case=False,regex=False))
]
print(get_stmt_and_file_count(convert_check))

[########################################] | 100% Completed | 8.88 ss
[########################################] | 100% Completed | 10.08 s
(1774, 134)


In [133]:
convert_check.head()

[########################################] | 100% Completed | 247.72 ms


c:\Users\geacu\anaconda3\envs\windows\lib\site-packages\dask\dataframe\core.py:7893: UserWarning: Insufficient elements for `head`. 5 elements requested, only 2 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
4230,015213,923,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: convert","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",convert,None,None,alter table t1 convert to character set utf8;
15206,016164,923,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: convert","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",convert,None,None,alter table t1 convert to character set utf8;


#### WITH

In [125]:
no_convert = nu_auto_incr_alter[
   (~nu_auto_incr_alter.found.str.contains('CONVERT',case=False,regex=False))
]
with_check = no_convert[
    (no_convert.found.str.contains('WITH',case=False,regex=False))
]
print(get_stmt_and_file_count(with_check))

[########################################] | 100% Completed | 8.47 ss
[########################################] | 100% Completed | 12.36 s
(1738, 220)


In [134]:
with_check.head()

[########################################] | 100% Completed | 330.50 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
7444,015491,2,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: WITH","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",WITH,None,None,"ALTER TABLE Conferences\nWITH CHECK ADD CONSTRAINT StudentDiscountBetweenZeroAndOne\n CHECK (StudentDiscount >= 0 AND StudentDiscount <= 1)\n\n\n-- Table: ConferenceDays\nCREATE TABLE ConferenceDays (\n ConferenceDayID int identity(1,1) NOT NULL,\n ConferenceID int NOT NULL,\n Date datetime NOT NULL,\n ParticipantsLimit int NOT NULL,\n Location nvarchar(60) NOT NULL,\n IsCancelled bit NOT NULL DEFAULT 0,\n CONSTRAINT ConferenceDays_pk PRIMARY KEY (ConferenceDayID)\n);"
7445,015491,3,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: WITH","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",WITH,None,None,ALTER TABLE ConferenceDays\nWITH CHECK ADD CONSTRAINT ParticipantsLimitGreaterThanZero\n CHECK (ParticipantsLimit > 0)\n\nALTER TABLE ConferenceDays ADD CONSTRAINT ConferenceDay_Conference_fk\n FOREIGN KEY (ConferenceID)\n REFERENCES Conferences (ConferenceID);
7448,015491,7,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: WITH","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",WITH,None,None,ALTER TABLE DayReservations\nWITH CHECK ADD CONSTRAINT StudentsNumberGreaterEqualsZero\n CHECK (StudentsNumber >= 0)\n\nALTER TABLE DayReservations\nWITH CHECK ADD CONSTRAINT ParticipantsNumberGreaterThanZero\n CHECK (ParticipantsNumber > 0)\n\nALTER TABLE DayReservations ADD CONSTRAINT DayReservation_Reservation_fk\n FOREIGN KEY (ReservationID)\n REFERENCES Reservations (ReservationID);
7450,015491,10,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: WITH","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",WITH,None,None,ALTER TABLE Workshops WITH CHECK ADD CONSTRAINT Workshop_WorkshopConferenceDay_fk\nFOREIGN KEY(ConferenceDayID) REFERENCES ConferenceDays (ConferenceDayID)\n\nALTER TABLE Workshops WITH CHECK ADD CONSTRAINT\n WorkshopsPriceGreaterEqualsZero\n CHECK ((Price >= (0)));
7451,015491,11,"Query parsing failed.\n\tsql parser error: Expected ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE, found: WITH","ADD, RENAME, PARTITION, SWAP or DROP after ALTER TABLE,",WITH,None,None,ALTER TABLE Workshops WITH CHECK ADD CONSTRAINT\n WorkshopsParticipantsLimitGreaterEqualsZero\n CHECK ((ParticipantsLimit >= (0)));


#### final

In [126]:
no_with = no_convert[
    (~no_convert.found.str.contains('WITH',case=False,regex=False))
]

In [127]:
print(get_stmt_and_file_count(no_with))

[########################################] | 100% Completed | 8.82 ss
[########################################] | 100% Completed | 7.94 ss
(6954, 1004)


In [287]:
to_write_down_hard.append(no_with.compute(scheduler='processes'))

[########################################] | 100% Completed | 7.30 ss


### Expected a list of columns in parentheses,

#### Found what

In [136]:
expected_list = expected_only[expected_only.expected == 'a list of columns in parentheses,']
expected_list['found'].value_counts().compute(scheduler='processes')

[########################################] | 100% Completed | 7.01 ss


KEY                                 201027
;                                   71341 
INDEX                               24234 
CLUSTERED                           7896  
,                                   3563  
                                    ...   
`fk_assigned_instrs_assignment`     1     
`fk_anuncios_contactos`             1     
`fk_acl_roles_parent`               1     
`fk_account_vlan_map__domain_id`    1     
�                                   1     
Name: found, Length: 7523, dtype: int64

In [137]:
print(get_stmt_and_file_count(expected_list))

[########################################] | 100% Completed | 8.38 ss
[########################################] | 100% Completed | 8.10 ss
(345329, 64331)


#### KEY

In [141]:
key_check = expected_list[
    (expected_list.found == 'KEY') &
    (expected_list.original.str.contains("KEY",case=False,regex=False))
]
print(get_stmt_and_file_count(key_check))

[########################################] | 100% Completed | 7.13 ss
[########################################] | 100% Completed | 6.34 ss
(201027, 41972)


In [139]:
key_check.head()

[########################################] | 100% Completed | 340.51 ms


file_id  statement_id                                                                                       parse_error                           expected found unterminated_what unterminated_where                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

#### semicolon

In [144]:
no_key = expected_list[
    ~((expected_list.found == 'KEY') &
    (expected_list.original.str.contains("KEY",case=False,regex=False)))
]

semicol_check = no_key[
    (no_key.found == ';') &
    (no_key.original.str.contains("ALTER TABLE",regex=False,case=False)) &
    (no_key.original.str.contains("foreign",regex=False,case=False))
]
print(get_stmt_and_file_count(semicol_check))

[########################################] | 100% Completed | 7.09 ss
[########################################] | 100% Completed | 6.71 ss
(71337, 3336)


In [143]:
semicol_check.head()

[########################################] | 100% Completed | 354.91 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
168,014783,3,"Query parsing failed.\n\tsql parser error: Expected a list of columns in parentheses, found: ;","a list of columns in parentheses,",;,None,None,alter table ACT_RU_IDENTITYLINK\n add constraint ACT_FK_ATHRZ_PROCEDEF\n foreign key (PROC_DEF_ID_)\n references ACT_RE_PROCDEF;
1434,014942,592,"Query parsing failed.\n\tsql parser error: Expected a list of columns in parentheses, found: ;","a list of columns in parentheses,",;,None,None,-- manually added foreign keys\n\nALTER TABLE ASP_CLIENTEXCEPTION ADD CONSTRAINT ADField_ASPClientException FOREIGN KEY (AD_Field_ID) REFERENCES AD_FIELD;
1435,014942,593,"Query parsing failed.\n\tsql parser error: Expected a list of columns in parentheses, found: ;","a list of columns in parentheses,",;,None,None,ALTER TABLE ASP_CLIENTEXCEPTION ADD CONSTRAINT ADForm_ASPClientException FOREIGN KEY (AD_Form_ID) REFERENCES AD_FORM;
1436,014942,594,"Query parsing failed.\n\tsql parser error: Expected a list of columns in parentheses, found: ;","a list of columns in parentheses,",;,None,None,ALTER TABLE ASP_CLIENTEXCEPTION ADD CONSTRAINT ADProcess_ASPClientException FOREIGN KEY (AD_Process_ID) REFERENCES AD_PROCESS;
1437,014942,595,"Query parsing failed.\n\tsql parser error: Expected a list of columns in parentheses, found: ;","a list of columns in parentheses,",;,None,None,ALTER TABLE ASP_CLIENTEXCEPTION ADD CONSTRAINT ADProcessPara_ASPClientExcepti FOREIGN KEY (AD_Process_Para_ID) REFERENCES AD_PROCESS_PARA;


#### INDEX

In [148]:
no_semicol = no_key[
    ~((no_key.found == ';') &
    (no_key.original.str.contains("ALTER TABLE",regex=False,case=False)) &
    (no_key.original.str.contains("foreign",regex=False,case=False)))
]

index_check = no_semicol[
    (no_semicol.found == 'INDEX') &
    (no_key.original.str.contains("CREATE",regex=False,case=False)) &
    (no_key.original.str.contains("UNIQUE",regex=False,case=False))
]
print(get_stmt_and_file_count(index_check))

[########################################] | 100% Completed | 7.63 ss
[########################################] | 100% Completed | 6.96 ss
(22316, 6097)


In [146]:
index_check.head()

[########################################] | 100% Completed | 252.59 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
555,014846,0,"Query parsing failed.\n\tsql parser error: Expected a list of columns in parentheses, found: INDEX","a list of columns in parentheses,",INDEX,None,None,"CREATE TABLE `hotel`.`customer` (\n `id` INT NOT NULL AUTO_INCREMENT ,\n `id_number` VARCHAR(50) NOT NULL ,\n `id_type` VARCHAR(45) NOT NULL ,\n `name` VARCHAR(45) NOT NULL ,\n `address` VARCHAR(45) NULL ,\n `phone` VARCHAR(45) NULL ,\n `age` INT NULL ,\n `birthday` DATETIME NULL ,\n `zip_code` VARCHAR(45) NULL ,\n PRIMARY KEY (`id`) ,\n UNIQUE INDEX `CUSTOMER_UQ` (`id_number` ASC, `id_type` ASC, `name` ASC) )\nCOMMENT = 'Table for customer data whom checkin checkout' ;"
560,014846,5,"Query parsing failed.\n\tsql parser error: Expected a list of columns in parentheses, found: INDEX","a list of columns in parentheses,",INDEX,None,None,"CREATE TABLE `hotel`.`event` (\n `id` INT NOT NULL ,\n `name` VARCHAR(45) NOT NULL ,\n PRIMARY KEY (`id`) ,\n UNIQUE INDEX `name_UNIQUE` (`name` ASC) )\nCOMMENT = 'Event Table' ;"
561,014846,6,"Query parsing failed.\n\tsql parser error: Expected a list of columns in parentheses, found: INDEX","a list of columns in parentheses,",INDEX,None,None,"CREATE TABLE `hotel`.`event_price` (\n `id` INT NOT NULL ,\n `id_event` INT NOT NULL ,\n `id_room_type` INT NOT NULL ,\n `price` DECIMAL(16,2) NOT NULL ,\n PRIMARY KEY (`id`) ,\n UNIQUE INDEX `EP_EVENT_ROOM_UQ` (`id_room_type` ASC, `id_event` ASC) ,\n INDEX `FK_EP_EVENT` (`id_event` ASC) ,\n INDEX `FK_EP_ROOM_TYPE` (`id_room_type` ASC) ,\n CONSTRAINT `FK_EP_EVENT`\n FOREIGN KEY (`id_event` )\n REFERENCES `hotel`.`event` (`id` )\n ON DELETE CASCADE\n ON UPDATE CASCADE,\n CONSTRAINT `FK_EP_ROOM_TYPE`\n FOREIGN KEY (`id_room_type` )\n REFERENCES `hotel`.`room_type` (`id` )\n ON DELETE CASCADE\n ON UPDATE CASCADE)\nCOMMENT = 'Price for the event that currently any' ;"
599,014851,21,"Query parsing failed.\n\tsql parser error: Expected a list of columns in parentheses, found: INDEX","a list of columns in parentheses,",INDEX,None,None,"/**\n@table transcript\n@desc Stores information about transcripts. Has seq_region_start, seq_region_end and seq_region_strand for faster retrieval and to allow storage independently of genes and exons.\nNote that a transcript is usually associated with a translation, but may not be, e.g. in the case of pseudogenes and RNA genes (those that code for RNA molecules).\n\n@column transcript_id Primary key, internal identifier.\n@column gene_id Foreign key references to the @link gene table.\n@column analysis_id Foreign key references to the @link analysis table.\n@column seq_region_id Foreign key references to the @link seq_region table.\n@column seq_region_start Sequence start position.\n@column seq_region_end Sequence end position.\n@column seq_region_strand Sequence region strand: 1 - forward; -1 - reverse.\n@column display_xref_id External reference for EnsEMBL web site. Foreign key references to the @link xref table.\n@column biotype Biotype, e.g. protein_coding.\n@column status Status, e.g.'KNOWN', 'NOVEL', 'PUTATIVE', 'PREDICTED', 'KNOWN_BY_PROJECTION', 'UNKNOWN'.\n@column description Transcript description.\n@column is_current\t\t Indicates a current transcript.\n@column canonical_translation_id Foreign key references to the @link translation table.\n@column stable_id\t\t Release-independent stable identifier.\n@column version \t Stable identifier version number.\n@column created_date \t Date created.\n@column modified_date \t Date modified.\n\n*/\n\n\nCREATE TABLE transcript (\n\n transcript_id INT(10) UNSIGNED NOT NULL AUTO_INCREMENT,\n gene_id INT(10) UNSIGNED,\n analysis_id SMALLINT UNSIGNED NOT NULL,\n seq_region_id INT(10) UNSIGNED NOT NULL,\n seq_region_start INT(10) UNSIGNED NOT NULL,\n seq_region_end INT(10) UNSIGNED NOT NULL,\n seq_region_strand TINYINT(2) NOT NULL,\n display_xref_id INT(10) UNSIGNED,\n biotype VARCHAR(40) NOT NULL,\n status ENUM('KNOWN', 'NOVE

#### CLUSTERED

In [151]:
no_index = no_semicol[
    ~((no_semicol.found == 'INDEX') &
    (no_key.original.str.contains("CREATE",regex=False,case=False)) &
    (no_key.original.str.contains("UNIQUE",regex=False,case=False)))
]
clus_check = no_index[
    (no_index.found == 'CLUSTERED') &
    (no_index.original.str.contains('[',regex=False,case=False))
]
print(get_stmt_and_file_count(clus_check))

[########################################] | 100% Completed | 7.23 ss
[########################################] | 100% Completed | 7.43 ss
(5737, 5083)


In [150]:
clus_check.head()

[########################################] | 100% Completed | 222.54 ms


c:\Users\geacu\anaconda3\envs\windows\lib\site-packages\dask\core.py:119: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
185,014795,0,"Query parsing failed.\n\tsql parser error: Expected a list of columns in parentheses, found: CLUSTERED","a list of columns in parentheses,",CLUSTERED,None,None,"CREATE TABLE [dbo].[Documento_Receber_Desconto] (\n [cd_documento_receber] INT NOT NULL,\n [cd_item_desconto] INT NOT NULL,\n [dt_desconto_documento] DATETIME NULL,\n [cd_banco] INT NULL,\n [vl_desconto_documento] FLOAT (53) NULL,\n [pc_desconto_documento] FLOAT (53) NULL,\n [nm_obs_desconto] VARCHAR (40) NULL,\n [cd_usuario] INT NULL,\n [dt_usuario] DATETIME NULL,\n [vl_liquido_documento] FLOAT (53) NULL,\n [cd_plano_financeiro] INT NULL,\n [cd_conta_banco] INT NULL,\n [cd_lancamento] INT NULL,\n [vl_juros_desconto] FLOAT (53) NULL,\n [vl_iof_desconto] FLOAT (53) NULL,\n [vl_custo_desconto] FLOAT (53) NULL,\n [qt_dia_desconto] FLOAT (53) NULL,\n [cd_bordero_desconto] INT NULL,\n CONSTRAINT [PK_Documento_Receber_Desconto] PRIMARY KEY CLUSTERED ([cd_documento_receber] ASC, [cd_item_desconto] ASC) WITH (FILLFACTOR = 90),\n CONSTRAINT [FK_Documento_Receber_Desconto_Banco] FOREIGN KEY ([cd_banco]) REFERENCES [dbo].[Banco] ([cd_banco]),\n CONSTRAINT [FK_Documento_Receber_Desconto_Documento_Receber] FOREIGN KEY ([cd_documento_receber]) REFERENCES [dbo].[Documento_Receber] ([cd_documento_receber]) ON DELETE CASCADE ON UPDATE CASCADE\n);"
207,014806,0,"Query parsing failed.\n\tsql parser error: Expected a list of columns in parentheses, found: CLUSTERED","a list of columns in parentheses,",CLUSTERED,None,None,"CREATE TABLE [HKP].[JobDescriptionDetail] (\n [Id] VARCHAR (10) NOT NULL,\n [JobDescriptionId] VARCHAR (10) NOT NULL,\n [FileName] VARCHAR (100) NOT NULL,\n [FileId] VARCHAR (50) NOT NULL,\n [AddedBy] VARCHAR (30) NOT NULL,\n [AddedDate] DATETIME NOT NULL,\n [AddedFromIP] VARCHAR (15) NOT NULL,\n [UpdatedBy] VARCHAR (30) NULL,\n [UpdatedDate] DATETIME NULL,\n [UpdatedFromIP] VARCHAR (15) NULL,\n CONSTRAINT [PK_JobDescriptionDetail] PRIMARY KEY CLUSTERED ([Id] ASC),\n CONSTRAINT [FK_JobDescriptionDetail_JobDescription] FOREIGN KEY ([JobDescriptionId]) REFERENCES [HKP].[JobDescription] ([Id])\n);"
1379,014932,0,"Query parsing failed.\n\tsql parser error: Expected a list of columns in parentheses, found: CLUSTERED","a list of columns in parentheses,",CLUSTERED,None,None,"CREATE TABLE [dbo].[Comissao_Fixa] (\n [cd_comissao_fixa] INT NOT NULL,\n [cd_pedido_venda] INT NULL,\n [cd_vendedor] INT NULL,\n [pc_comissao_fixa] FLOAT (53) NULL,\n [vl_comissao_fixa] MONEY NULL,\n [nm_obs_comissao_fixa] VARCHAR (40) NULL,\n [cd_usuario] INT NULL,\n [dt_usuario] DATETIME NULL,\n CONSTRAINT [PK_Comissao_Fixa] PRIMARY KEY CLUSTERED ([cd_comissao_fixa] ASC) WITH (FILLFACTOR = 90),\n CONSTRAINT [FK_Comissao_Fixa_Pedido_Venda] FOREIGN KEY ([cd_pedido_venda]) REFERENCES [dbo].[Pedido_Venda] ([cd_pedido_venda]),\n CONSTRAINT [FK_Comissao_Fixa_Vendedor] FOREIGN KEY ([cd_vendedor]) REFERENCES [dbo].[Vendedor] ([cd_vendedor])\n);"
3559,015180,0,"Query parsing failed.\n\tsql parser error: Expected a list of columns in parentheses, found: CLUSTERED","a list of columns in parentheses,",CLUSTERED,None,None,"CREATE TABLE [dbo].[Contacet] (\n [Id] INT NOT NULL,\n [UserId] INT NOT NULL,\n [ExtendType] NVARCHAR (64) NULL,\n [ExtendCode] NVARCHAR (64) NULL,\n [ImageId] INT NULL,\n [DisplayName] NVARCHAR (64) NULL,\n [FirstName] NVARCHAR (64) NULL,\n [FistNameExt] NVARCHAR (64) NULL,\n [MiddleName] NVARCHAR (64) NULL,\n [MiddleNameExt] NVARCHAR (64) NULL,\n [LastName] NVARCHAR (64) NULL,\n [LastNameExt] NVARCHAR (64) NULL,\n [Prefix] NVARCHAR (64) NULL,\n [Suffix] NVARCHAR (64) NULL,\n [NickName] NVARCHAR (64) NULL,\n [Birthday] DATETIME NULL,\n CONSTRAINT [PK_CONTACET] PRIMARY KEY CLUSTERED ([Id] ASC),\n CONSTRAINT [FK_CONTACET_FK_USERRE_USERRESO] FOREIGN KEY ([UserId]) REFERENCES [dbo].[UserResource] ([Id]) ON DELETE CASCADE ON UPDATE CASCADE\n);"
4456,015218,0,"Query parsing failed.\n\tsql parser 

#### final

In [152]:
no_clus = no_index[
    ~((no_index.found == 'CLUSTERED') &
    (no_index.original.str.contains('[',regex=False,case=False)))
]

In [153]:
print(get_stmt_and_file_count(no_clus))

[########################################] | 100% Completed | 7.62 ss
[########################################] | 100% Completed | 8.09 ss
(44912, 10084)


In [288]:
to_write_down_hard.append(no_clus.compute(scheduler='processes'))

[########################################] | 100% Completed | 7.22 ss


### Expected "',' or ')' after column definition,"

In [186]:
expected_after_col_def = expected_only[expected_only.expected == "',' or ')' after column definition,"]
expected_after_col_def['found'].value_counts().compute(scheduler='processes')

[########################################] | 100% Completed | 8.86 ss


COLLATE             112243
USING               42215 
IDENTITY            35073 
unsigned            18569 
(                   18518 
                    ...   
`log`               1     
`loan_bank_code`    1     
`lastlogin`         1     
`jobName`           1     
�                   1     
Name: found, Length: 1720, dtype: int64

In [187]:
print(get_stmt_and_file_count(expected_after_col_def))

[########################################] | 100% Completed | 7.04 ss
[########################################] | 100% Completed | 6.78 ss
(329127, 72209)


#### COLLATE

In [190]:
collate_check = expected_after_col_def[
    expected_after_col_def.found == 'COLLATE'
]
print(get_stmt_and_file_count(collate_check))

[########################################] | 100% Completed | 6.57 ss
[########################################] | 100% Completed | 7.10 ss
(112243, 18346)


In [189]:
collate_check.head()

[########################################] | 100% Completed | 228.01 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
371,014834,25,"Query parsing failed.\n\tsql parser error: Expected ',' or ')' after column definition, found: COLLATE","',' or ')' after column definition,",COLLATE,None,None,"-- --------------------------------------------------------\n\n--\n-- Table structure for table `poruka`\n--\n\nCREATE TABLE `poruka` (\n `porukaid` int(11) NOT NULL,\n `ime` varchar(30) CHARACTER SET utf8 COLLATE utf8_unicode_ci NOT NULL,\n `prezime` varchar(30) CHARACTER SET utf8 COLLATE utf8_unicode_ci NOT NULL,\n `email` varchar(30) CHARACTER SET utf8 COLLATE utf8_unicode_ci NOT NULL,\n `poruka` text CHARACTER SET utf8 COLLATE utf8_unicode_ci NOT NULL,\n `vreme` datetime NOT NULL DEFAULT current_timestamp(),\n `procitano` tinyint(4) NOT NULL\n) ENGINE=MyISAM DEFAULT CHARSET=latin1;"
665,014853,70,"Query parsing failed.\n\tsql parser error: Expected ',' or ')' after column definition, found: COLLATE","',' or ')' after column definition,",COLLATE,None,None,"CREATE TABLE `settings` (\n `settings_id` int(11) NOT NULL AUTO_INCREMENT,\n `type` longtext CHARACTER SET utf8 COLLATE utf8_unicode_ci NOT NULL,\n `description` longtext CHARACTER SET utf8 COLLATE utf8_unicode_ci NOT NULL,\n PRIMARY KEY (`settings_id`)\n) ENGINE=MyISAM DEFAULT CHARSET=latin1;"
1286,014920,3,"Query parsing failed.\n\tsql parser error: Expected ',' or ')' after column definition, found: COLLATE","',' or ')' after column definition,",COLLATE,None,None,"CREATE TABLE `building_type` (\n `bt_id` int(11) NOT NULL AUTO_INCREMENT,\n `bt_building_type` varchar(255) CHARACTER SET utf16 COLLATE utf16_bin NULL DEFAULT NULL,\n `bt_is_active` char(1) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT 'Y',\n `last_user` varchar(255) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL,\n `last_datetime` datetime(0) NULL DEFAULT NULL,\n PRIMARY KEY (`bt_id`) USING BTREE\n) ENGINE = InnoDB AUTO_INCREMENT = 19 CHARACTER SET = utf8 COLLATE = utf8_general_ci ROW_FORMAT = Compact;"
1287,014920,8,"Query parsing failed.\n\tsql parser error: Expected ',' or ')' after column definition, found: COLLATE","',' or ')' after column definition,",COLLATE,None,None,"CREATE TABLE `item_list` (\n `il_id` int(11) NOT NULL AUTO_INCREMENT,\n `il_item_name` varchar(255) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL,\n `il_un_id` int(11) NULL DEFAULT NULL,\n `il_is_active` char(1) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT 'Y',\n `last_user` varchar(255) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL,\n `last_datetime` datetime(0) NULL DEFAULT NULL,\n PRIMARY KEY (`il_id`) USING BTREE\n) ENGINE = InnoDB AUTO_INCREMENT = 17 CHARACTER SET = utf8 COLLATE = utf8_general_ci ROW_FORMAT = Compact;"
1288,014920,19,"Query parsing failed.\n\tsql parser error: Expected ',' or ')' after column definition, found: COLLATE","',' or ')' after column definition,",COLLATE,None,None,"CREATE TABLE `item_rab` (\n `ir_id` int(11) NOT NULL AUTO_INCREMENT,\n `ir_item_name` varchar(255) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL,\n `ir_un_id` int(11) NULL DEFAULT NULL,\n `ir_seq` int(11) NULL DEFAULT NULL,\n `ir_is_active` char(1) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT 'Y',\n `last_user` varchar(255) CHARACTER SET utf8 COLLATE utf8_general_ci NULL DEFAULT NULL,\n `last_datetime` datetime(0) NULL DEFAULT NULL,\n PRIMARY KEY (`ir_id`) USING BTREE\n) ENGINE = InnoDB AUTO_INCREMENT = 17 CHARACTER SET = utf8 COLLATE = utf8_general_ci ROW_FORMAT = Compact;"


#### USING

In [193]:
no_collate = expected_after_col_def[
    ~(expected_after_col_def.found == 'COLLATE')
]
using_check = no_collate[
    no_collate.found == 'USING'
]
print(get_stmt_and_file_count(using_check))

[########################################] | 100% Completed | 7.43 ss
[########################################] | 100% Completed | 6.81 ss
(42215, 8967)


In [192]:
using_check.head()

[########################################] | 100% Completed | 115.56 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
682,014854,793,"Query parsing failed.\n\tsql parser error: Expected ',' or ')' after column definition, found: USING","',' or ')' after column definition,",USING,None,None,"CREATE TABLE `fa_command` (\n `id` int(10) unsigned NOT NULL AUTO_INCREMENT COMMENT 'ID',\n `type` varchar(30) NOT NULL DEFAULT '' COMMENT '类型',\n `params` varchar(1500) NOT NULL DEFAULT '' COMMENT '参数',\n `command` varchar(1500) NOT NULL DEFAULT '' COMMENT '命令',\n `content` text COMMENT '返回结果',\n `executetime` int(10) unsigned NOT NULL DEFAULT '0' COMMENT '执行时间',\n `createtime` int(10) unsigned NOT NULL DEFAULT '0' COMMENT '创建时间',\n `updatetime` int(10) unsigned NOT NULL DEFAULT '0' COMMENT '更新时间',\n `status` enum('successed','failured') NOT NULL DEFAULT 'failured' COMMENT '状态',\n PRIMARY KEY (`id`) USING BTREE\n) ENGINE=InnoDB AUTO_INCREMENT=54 DEFAULT CHARSET=utf8 ROW_FORMAT=COMPACT COMMENT='在线命令表';"
684,014854,879,"Query parsing failed.\n\tsql parser error: Expected ',' or ')' after column definition, found: USING","',' or ')' after column definition,",USING,None,None,"CREATE TABLE `fa_ems` (\n `id` int(10) unsigned NOT NULL AUTO_INCREMENT COMMENT 'ID',\n `event` varchar(30) NOT NULL DEFAULT '' COMMENT '事件',\n `email` varchar(100) NOT NULL DEFAULT '' COMMENT '邮箱',\n `code` varchar(10) NOT NULL DEFAULT '' COMMENT '验证码',\n `times` int(10) unsigned NOT NULL DEFAULT '0' COMMENT '验证次数',\n `ip` varchar(30) NOT NULL DEFAULT '' COMMENT 'IP',\n `createtime` int(10) unsigned DEFAULT '0' COMMENT '创建时间',\n PRIMARY KEY (`id`) USING BTREE\n) ENGINE=InnoDB DEFAULT CHARSET=utf8 ROW_FORMAT=COMPACT COMMENT='邮箱验证码表';"
691,014854,911,"Query parsing failed.\n\tsql parser error: Expected ',' or ')' after column definition, found: USING","',' or ')' after column definition,",USING,None,None,"CREATE TABLE `fa_version` (\n `id` int(11) NOT NULL AUTO_INCREMENT COMMENT 'ID',\n `oldversion` varchar(30) NOT NULL DEFAULT '' COMMENT '旧版本号',\n `newversion` varchar(30) NOT NULL DEFAULT '' COMMENT '新版本号',\n `packagesize` varchar(30) NOT NULL DEFAULT '' COMMENT '包大小',\n `content` varchar(500) NOT NULL DEFAULT '' COMMENT '升级内容',\n `downloadurl` varchar(255) NOT NULL DEFAULT '' COMMENT '下载地址',\n `enforce` tinyint(1) unsigned NOT NULL DEFAULT '0' COMMENT '强制更新',\n `createtime` int(10) NOT NULL DEFAULT '0' COMMENT '创建时间',\n `updatetime` int(10) unsigned NOT NULL DEFAULT '0' COMMENT '更新时间',\n `weigh` int(10) NOT NULL DEFAULT '0' COMMENT '权重',\n `status` varchar(30) NOT NULL DEFAULT '' COMMENT '状态',\n PRIMARY KEY (`id`) USING BTREE\n) ENGINE=InnoDB AUTO_INCREMENT=2 DEFAULT CHARSET=utf8 ROW_FORMAT=COMPACT COMMENT='版本表';"
1142,014909,52,"Query parsing failed.\n\tsql parser error: Expected ',' or ')' after column definition, found: USING","',' or ')' after column definition,",USING,None,None,"CREATE TABLE `Classes` (\n `id` int(11) NOT NULL AUTO_INCREMENT,\n `app_id` int(11) NOT NULL,\n `class` varchar(191) COLLATE utf8mb4_bin NOT NULL,\n PRIMARY KEY (`id`),\n KEY `class_idx` (`class`) USING HASH,\n KEY `app_id_idx` (`app_id`) USING HASH,\n CONSTRAINT `Classes_ibfk_1` FOREIGN KEY (`app_id`) REFERENCES `Applications` (`id`) ON DELETE CASCADE\n) ENGINE=InnoDB AUTO_INCREMENT=166 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_bin;"
1174,014909,161,"Query parsing failed.\n\tsql parser error: Expected ',' or ')' after column definition, found: USING","',' or ')' after column definition,",USING,None,None,"CREATE TABLE `IFActions` (\n `id` int(11) NOT NULL AUTO_INCREMENT,\n `filter_id` int(11) NOT NULL,\n `action` int(11) NOT NULL,\n PRIMARY KEY (`id`),\n KEY `filter_id` (`filter_id`),\n KEY `action_idx` (`action`) USING HASH,\n CONSTRAINT `IFActions_ibfk_1` FOREIGN KEY (`filter_id`) REFERENCES `IntentFilters` (`id`) ON DELETE CASCADE,\n CONSTRAINT `IFActions_ibfk_2` FOREIGN KEY (`action`) REFERENCES `ActionStrings` (`id`)\n) ENGINE=InnoDB AUTO_INCREMENT=216 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_bin;"


#### IDENTITY

In [196]:
no_us = no_collate[
    ~(no_collate.found == 'USING')
]
id_check = no_us[
    no_us.found == 'IDENTITY'
]
print(get_stmt_and_file_count(id_check))

[########################################] | 100% Completed | 7.00 ss
[########################################] | 100% Completed | 6.43 ss
(35073, 15549)


In [195]:
id_check.head()

[########################################] | 100% Completed | 244.68 ms


file_id  statement_id                                                                                              parse_error                             expected     found unterminated_what unterminated_where                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

#### final

In [199]:
no_id = no_us[
    ~(no_us.found == 'IDENTITY')
]

In [200]:
print(get_stmt_and_file_count(no_id))

[########################################] | 100% Completed | 7.07 ss
[########################################] | 100% Completed | 6.29 ss
(139596, 36866)


In [289]:
to_write_down_hard.append(no_id.compute(scheduler='processes'))

[########################################] | 100% Completed | 7.23 ss


### Expected "),"

In [197]:
expected_parant = expected_only[expected_only.expected == "),"]
expected_parant['found'].value_counts().compute(scheduler='processes')


[########################################] | 100% Completed | 7.04 ss


ASC                  44310
{                    20803
(                    18605
,                    13888
'0'                  9183 
                     ...  
Id                   1    
ISMAN                1    
INVOICE_REFERENCE    1    
INTO                 1    
：                    1    
Name: found, Length: 1673, dtype: int64

In [198]:
print(get_stmt_and_file_count(expected_parant))

[########################################] | 100% Completed | 6.60 ss
[########################################] | 100% Completed | 6.25 ss
(142034, 21395)


#### ASC

In [204]:
asc_check = expected_parant[
    (expected_parant.found == 'ASC')&
    (expected_parant.original.str.contains("INDEX",case=False,regex=False))
]
print(get_stmt_and_file_count(asc_check))

[########################################] | 100% Completed | 7.15 ss
[########################################] | 100% Completed | 6.35 ss
(42686, 6681)


In [202]:
asc_check.head()

[########################################] | 100% Completed | 244.60 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
556,014846,1,"Query parsing failed.\n\tsql parser error: Expected ), found: ASC","),",ASC,None,None,"CREATE TABLE `hotel`.`facility` (\n `id` INT NOT NULL AUTO_INCREMENT ,\n `name` VARCHAR(45) NOT NULL ,\n `default_price` DECIMAL(16,2) NOT NULL ,\n PRIMARY KEY (`id`) ,\n INDEX `NAME_IDX` (`name` ASC) )\nCOMMENT = 'Master table for facility that exist' ;"
557,014846,2,"Query parsing failed.\n\tsql parser error: Expected ), found: ASC","),",ASC,None,None,"CREATE TABLE `hotel`.`purchase` (\n `id` INT NOT NULL AUTO_INCREMENT ,\n `name` VARCHAR(45) NOT NULL ,\n `default_price` DECIMAL(16,2) NOT NULL ,\n PRIMARY KEY (`id`) ,\n INDEX `NAME_IDX` (`name` ASC) )\nCOMMENT = 'Master table for item available for purchase' ;"
558,014846,3,"Query parsing failed.\n\tsql parser error: Expected ), found: ASC","),",ASC,None,None,"CREATE TABLE `hotel`.`room_type` (\n `id` INT NOT NULL ,\n `name` VARCHAR(45) NOT NULL ,\n `description` VARCHAR(150) NULL ,\n PRIMARY KEY (`id`) ,\n INDEX `RT_NAME_IDX` (`name` ASC) );"
559,014846,4,"Query parsing failed.\n\tsql parser error: Expected ), found: ASC","),",ASC,None,None,"CREATE TABLE `hotel`.`room` (\n `id` INT NOT NULL ,\n `id_room_type` INT ,\n `room_number` INT NOT NULL ,\n `room_floor` INT NOT NULL ,\n `name` VARCHAR(15) NULL ,\n `status` INT NULL ,\n `description` VARCHAR(150) NULL ,\n PRIMARY KEY (`id`) ,\n INDEX `R_NUMBER_IDX` (`room_number` ASC) ,\n INDEX `R_FLOOR_IDX` (`room_floor` ASC) ,\n INDEX `FK_R_ROOM_TYPE` (`id_room_type` ASC) ,\n CONSTRAINT `FK_R_ROOM_TYPE`\n FOREIGN KEY (`id_room_type` )\n REFERENCES `hotel`.`room_type` (`id` )\n ON DELETE SET NULL\n ON UPDATE CASCADE)\nCOMMENT = 'Table for facility transaction such as blanket, extra bed' /* comment truncated */ \n;"
562,014846,7,"Query parsing failed.\n\tsql parser error: Expected ), found: ASC","),",ASC,None,None,"CREATE TABLE `hotel`.`transaction` (\n `id` BIGINT NOT NULL AUTO_INCREMENT ,\n `id_customer` INT NOT NULL ,\n `status` INT NOT NULL COMMENT 'transaction status' ,\n `check_in_time` DATETIME NULL ,\n `check_in_by` INT NULL ,\n `check_out_time` DATETIME NULL ,\n `check_out_by` INT NULL ,\n `status_id_card` VARCHAR(45) NULL ,\n `id_room` INT NOT NULL ,\n `id_event` INT NOT NULL ,\n `room_price` DECIMAL(16,2) NOT NULL ,\n `total_fee` DECIMAL(16,2) NULL ,\n `current_fee` DECIMAL(16,2) NULL ,\n `paid_fee` DECIMAL(16,2) NULL ,\n `unpaid_fee` DECIMAL(16,2) NULL ,\n `created_date` DATETIME NOT NULL ,\n `created_by` INT NOT NULL ,\n `modified_date` DATETIME NOT NULL ,\n `modified_by` INT NOT NULL ,\n `data_customer` VARCHAR(250) NULL ,\n PRIMARY KEY (`id`) ,\n INDEX `FK_T_CUSTOMER` (`id_customer` ASC) ,\n INDEX `FK_T_ROOM` (`id_room` ASC) ,\n INDEX `FK_T_EVENT` (`id_event` ASC) ,\n CONSTRAINT `FK_T_CUSTOMER`\n FOREIGN KEY (`id_customer` )\n REFERENCES `hotel`.`customer` (`id` )\n ON DELETE NO ACTION\n ON UPDATE NO ACTION,\n CONSTRAINT `FK_T_ROOM`\n FOREIGN KEY (`id_room` )\n REFERENCES `hotel`.`room` (`id` )\n ON DELETE NO ACTION\n ON UPDATE CASCADE,\n CONSTRAINT `FK_T_EVENT`\n FOREIGN KEY (`id_event` )\n REFERENCES `hotel`.`event` (`id` )\n ON DELETE NO ACTION\n ON UPDATE CASCADE)\nCOMMENT = 'Transactional event' ;"


#### {

In [207]:
no_asc = expected_parant[
    ~((expected_parant.found == 'ASC')&
    (expected_parant.original.str.contains("INDEX",case=False,regex=False)))
]
thing_check = no_asc[
    (no_asc.found =='{')
]
print(get_stmt_and_file_count(thing_check))

[########################################] | 100% Completed | 7.58 ss
[########################################] | 100% Completed | 7.58 ss
(20803, 553)


In [206]:
thing_check.head()

[########################################] | 100% Completed | 224.23 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
229,014813,1,"Query parsing failed.\n\tsql parser error: Expected ), found: {","),",{,None,None,"INSERT INTO APPLICATIONS(ID, NAME, VERSION) VALUES($APP_ID{}, $APP_NAME{}, $APP_VERSION{});"
230,014813,4,"Query parsing failed.\n\tsql parser error: Expected ), found: {","),",{,None,None,"INSERT INTO ROLES(ID, NAME, PERMISSIONS) VALUES('0', 'Administrator role', $FILE{/com/openbravo/pos/templates/Role.Administrator.xml} );"
231,014813,5,"Query parsing failed.\n\tsql parser error: Expected ), found: {","),",{,None,None,"INSERT INTO ROLES(ID, NAME, PERMISSIONS) VALUES('1', 'Manager role', $FILE{/com/openbravo/pos/templates/Role.Manager.xml} );"
232,014813,6,"Query parsing failed.\n\tsql parser error: Expected ), found: {","),",{,None,None,"INSERT INTO ROLES(ID, NAME, PERMISSIONS) VALUES('2', 'Employee role', $FILE{/com/openbravo/pos/templates/Role.Employee.xml} );"
233,014813,7,"Query parsing failed.\n\tsql parser error: Expected ), found: {","),",{,None,None,"INSERT INTO ROLES(ID, NAME, PERMISSIONS) VALUES('3', 'Guest role', $FILE{/com/openbravo/pos/templates/Role.Guest.xml} );"


#### final

In [208]:
no_thing = no_asc[
    ~(no_asc.found =='{')
]

In [209]:
print(get_stmt_and_file_count(no_thing))

[########################################] | 100% Completed | 8.36 ss
[########################################] | 100% Completed | 7.11 ss
(78545, 14419)


In [290]:
to_write_down_hard.append(no_thing.compute(scheduler='processes'))

[########################################] | 100% Completed | 7.03 ss


### Expected "an object type after CREATE,"

#### WHAT

In [210]:
expected_obj_type = expected_only[expected_only.expected == "an object type after CREATE,"]
expected_obj_type['found'].value_counts().compute(scheduler='processes')


[########################################] | 100% Completed | 8.25 ss


DEFINER                                    14132
TRIGGER                                    13954
TYPE                                       5108 
cached                                     5024 
USER                                       4732 
PROCEDURE                                  3799 
ALGORITHM                                  3742 
EXTENSION                                  2376 
RULE                                       2259 
FUNCTION                                   1679 
FOREIGN                                    1629 
user                                       1451 
DOMAIN                                     1443 
CACHED                                     1259 
trigger                                    1128 
GENERATOR                                  973  
SYNONYM                                    921  
domain                                     901  
rule                                       732  
PUBLIC                                     625  
synonym             

In [211]:
print(get_stmt_and_file_count(expected_obj_type))

[########################################] | 100% Completed | 6.09 ss
[########################################] | 100% Completed | 6.92 ss
(76579, 16115)


#### lots    

In [220]:
lots_check = expected_obj_type[
    (expected_obj_type.found == 'DEFINER') | 
    (expected_obj_type.found == 'TRIGGER') | 
    (expected_obj_type.found == 'TYPE') | 
    (expected_obj_type.found == 'cached') | 
    (expected_obj_type.found == 'USER') | 
    (expected_obj_type.found == 'PROCEDURE') | 
    (expected_obj_type.found == 'EXTENSION') | 
    (expected_obj_type.found == 'ALGORITHM') | 
    (expected_obj_type.found == 'FUNCTION') | 
    (expected_obj_type.found == 'RULE')
]
print(get_stmt_and_file_count(lots_check))

[########################################] | 100% Completed | 13.04 s
[########################################] | 100% Completed | 12.65 s
(56805, 12449)


In [218]:
#expected_obj_type[expected_obj_type.found == 'EXTENSION'].head()

#### final

In [221]:
no_lots = expected_obj_type[
    ~((expected_obj_type.found == 'DEFINER') | 
    (expected_obj_type.found == 'TRIGGER') | 
    (expected_obj_type.found == 'TYPE') | 
    (expected_obj_type.found == 'cached') | 
    (expected_obj_type.found == 'USER') | 
    (expected_obj_type.found == 'PROCEDURE') | 
    (expected_obj_type.found == 'EXTENSION') | 
    (expected_obj_type.found == 'ALGORITHM') | 
    (expected_obj_type.found == 'FUNCTION') | 
    (expected_obj_type.found == 'RULE'))
]

In [222]:
print(get_stmt_and_file_count(no_lots))

[########################################] | 100% Completed | 9.45 ss
[########################################] | 100% Completed | 9.06 ss
(19774, 4380)


In [291]:
to_write_down_hard.append(no_lots.compute(scheduler='processes'))

[########################################] | 100% Completed | 7.03 ss


### Expected "SELECT, VALUES, or a subquery in the query body,"

In [223]:
expected_select = expected_only[expected_only.expected == "SELECT, VALUES, or a subquery in the query body,"]
expected_select['found'].value_counts().compute(scheduler='processes')


[########################################] | 100% Completed | 8.75 ss


INTO                   20731
NULL                   8379 
value                  7482 
VALUE                  6247 
SET                    3873 
                       ...  
'Yaara Silly Silly'    1    
'Yaariyan'             1    
'Zed Plus'             1    
'Zid'                  1    
폐기예정                   1    
Name: found, Length: 3402, dtype: int64

In [224]:
print(get_stmt_and_file_count(expected_select))

[########################################] | 100% Completed | 8.64 ss
[########################################] | 100% Completed | 8.88 ss
(70698, 3797)


#### INTO

In [228]:
into_check = expected_select[
    (expected_select.found == 'INTO') &
    (expected_select.original.str.contains("INSERT IGNORE INTO",regex=False,case=False))
]
print(get_stmt_and_file_count(into_check))

[########################################] | 100% Completed | 7.64 ss
[########################################] | 100% Completed | 8.54 ss
(19466, 1488)


In [226]:
into_check.head()

[########################################] | 100% Completed | 293.02 ms


file_id  statement_id                                                                                                       parse_error                                          expected found unterminated_what unterminated_where                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

#### final

In [229]:
no_into = expected_select[
    ~((expected_select.found == 'INTO') &
    (expected_select.original.str.contains("INSERT IGNORE INTO",regex=False,case=False)))
]

In [230]:
print(get_stmt_and_file_count(no_into))

[########################################] | 100% Completed | 9.19 ss
[########################################] | 100% Completed | 8.09 ss
(51232, 2358)


In [292]:
to_write_down_hard.append(no_into.compute(scheduler='processes'))

[########################################] | 100% Completed | 7.24 ss


### Expected "TABLE, VIEW, INDEX, ROLE, SCHEMA, FUNCTION, STAGE or SEQUENCE after DROP,"

#### WHAT

In [232]:
expected_after_drop = expected_only[expected_only.expected == "TABLE, VIEW, INDEX, ROLE, SCHEMA, FUNCTION, STAGE or SEQUENCE after DROP,"]
expected_after_drop['found'].value_counts().compute(scheduler='processes')

[########################################] | 100% Completed | 7.74 ss


DATABASE               11765
PROCEDURE              11305
TRIGGER                4743 
database               4312 
PREPARE                3054 
procedure              2495 
USER                   1691 
TYPE                   950  
trigger                589  
user                   460  
EVENT                  278  
domain                 238  
public                 216  
class                  168  
TEMPORARY              154  
FOREIGN                154  
type                   140  
tables                 131  
EXTENSION              127  
TABLESPACE             108  
PUBLIC                 87   
rule                   81   
AGGREGATE              70   
temporary              70   
DOMAIN                 65   
SERVER                 64   
ALL                    61   
RULE                   58   
GENERATOR              53   
Database               50   
SYNONYM                43   
parent                 43   
MATERIALIZED           37   
child                  36   
foreign       

In [233]:
print(get_stmt_and_file_count(expected_after_drop))

[########################################] | 100% Completed | 7.82 ss
[########################################] | 100% Completed | 7.76 ss
(44497, 19697)


#### Chk

In [234]:
chk_drop = expected_after_drop[
    (expected_after_drop.found == 'DATABASE') |
    (expected_after_drop.found == 'PROCEDURE') |
    (expected_after_drop.found == 'TRIGGER') |
    (expected_after_drop.found == 'database') |
    (expected_after_drop.found == 'PREPARE') |
    (expected_after_drop.found == 'procedure') |
    (expected_after_drop.found == 'TYPE') |
    (expected_after_drop.found == 'USER') 
]
print(get_stmt_and_file_count(chk_drop))

[########################################] | 100% Completed | 10.32 s
[########################################] | 100% Completed | 10.45 s
(40315, 18550)


In [235]:
chk_drop.head(10)

[########################################] | 100% Completed | 270.29 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
0,014743,0,"Query parsing failed.\n\tsql parser error: Expected TABLE, VIEW, INDEX, ROLE, SCHEMA, FUNCTION, STAGE or SEQUENCE after DROP, found: DATABASE","TABLE, VIEW, INDEX, ROLE, SCHEMA, FUNCTION, STAGE or SEQUENCE after DROP,",DATABASE,None,None,DROP DATABASE IF EXISTS INVENTORY;
115,014772,0,"Query parsing failed.\n\tsql parser error: Expected TABLE, VIEW, INDEX, ROLE, SCHEMA, FUNCTION, STAGE or SEQUENCE after DROP, found: database","TABLE, VIEW, INDEX, ROLE, SCHEMA, FUNCTION, STAGE or SEQUENCE after DROP,",database,None,None,drop database if exists Auto_servis;
169,014786,0,"Query parsing failed.\n\tsql parser error: Expected TABLE, VIEW, INDEX, ROLE, SCHEMA, FUNCTION, STAGE or SEQUENCE after DROP, found: database","TABLE, VIEW, INDEX, ROLE, SCHEMA, FUNCTION, STAGE or SEQUENCE after DROP,",database,None,None,drop database if exists baitap_phantich;
290,014816,2,"Query parsing failed.\n\tsql parser error: Expected TABLE, VIEW, INDEX, ROLE, SCHEMA, FUNCTION, STAGE or SEQUENCE after DROP, found: database","TABLE, VIEW, INDEX, ROLE, SCHEMA, FUNCTION, STAGE or SEQUENCE after DROP,",database,None,None,drop database proyecto_tic1v2;
740,014855,277,"Query parsing failed.\n\tsql parser error: Expected TABLE, VIEW, INDEX, ROLE, SCHEMA, FUNCTION, STAGE or SEQUENCE after DROP, found: PROCEDURE","TABLE, VIEW, INDEX, ROLE, SCHEMA, FUNCTION, STAGE or SEQUENCE after DROP,",PROCEDURE,None,None,-- BELOW ARE EXTRA PROCEDURES AND TABLES THAT YOU WILL NEED FOR THE PROJECT\n\nDROP PROCEDURE IF EXISTS add_event_week;
744,014855,283,"Query parsing failed.\n\tsql parser error: Expected TABLE, VIEW, INDEX, ROLE, SCHEMA, FUNCTION, STAGE or SEQUENCE after DROP, found: PROCEDURE","TABLE, VIEW, INDEX, ROLE, SCHEMA, FUNCTION, STAGE or SEQUENCE after DROP,",PROCEDURE,None,None,DROP PROCEDURE IF EXISTS add_daily;
748,014855,289,"Query parsing failed.\n\tsql parser error: Expected TABLE, VIEW, INDEX, ROLE, SCHEMA, FUNCTION, STAGE or SEQUENCE after DROP, found: PROCEDURE","TABLE, VIEW, INDEX, ROLE, SCHEMA, FUNCTION, STAGE or SEQUENCE after DROP,",PROCEDURE,None,None,DROP PROCEDURE IF EXISTS add_action;
900,014882,0,"Query parsing failed.\n\tsql parser error: Expected TABLE, VIEW, INDEX, ROLE, SCHEMA, FUNCTION, STAGE or SEQUENCE after DROP, found: database","TABLE, VIEW, INDEX, ROLE, SCHEMA, FUNCTION, STAGE or SEQUENCE after DROP,",database,None,None,drop database if exists dragonsystem;
929,014891,0,"Query parsing failed.\n\tsql parser error: Expected TABLE, VIEW, INDEX, ROLE, SCHEMA, FUNCTION, STAGE or SEQUENCE after DROP, found: DATABASE","TABLE, VIEW, INDEX, ROLE, SCHEMA, FUNCTION, STAGE or SEQUENCE after DROP,",DATABASE,None,None,-- DROP database ebarrotes;\nDROP DATABASE IF EXISTS `ebarrotes`;
942,014899,0,"Query parsing failed.\n\tsql parser error: Expected TABLE, VIEW, INDEX, ROLE, SCHEMA, FUNCTION, STAGE or SEQUENCE after DROP, found: DATABASE","TABLE, VIEW, INDEX, ROLE, SCHEMA, FUNCTION, STAGE or SEQUENCE after DROP,",DATABASE,None,None,--\n-- Name: ${ databaseName }; Type: DATABASE\n--\n\nDROP DATABASE IF EXISTS ${ databaseName };


#### final

In [236]:
no_chk_drop = expected_after_drop[
    ~((expected_after_drop.found == 'DATABASE') |
    (expected_after_drop.found == 'PROCEDURE') |
    (expected_after_drop.found == 'TRIGGER') |
    (expected_after_drop.found == 'database') |
    (expected_after_drop.found == 'PREPARE') |
    (expected_after_drop.found == 'procedure') |
    (expected_after_drop.found == 'TYPE') |
    (expected_after_drop.found == 'USER')) 
]

In [237]:
print(get_stmt_and_file_count(no_chk_drop))

[########################################] | 100% Completed | 8.97 ss
[########################################] | 100% Completed | 8.27 ss
(4182, 1600)


In [293]:
to_write_down_hard.append(no_chk_drop.compute(scheduler='processes'))

[########################################] | 100% Completed | 6.80 ss


### Expected "one of TABLE or INDEX,"

In [238]:
expected_one_of = expected_only[expected_only.expected == "one of TABLE or INDEX,"]
expected_one_of['found'].value_counts().compute(scheduler='processes')


[########################################] | 100% Completed | 7.67 ss


SEQUENCE                 32571
DATABASE                 1712 
SCHEMA                   789  
TYPE                     610  
USER                     597  
sequence                 479  
SESSION                  320  
FUNCTION                 288  
session                  217  
IGNORE                   200  
ROLE                     189  
FOREIGN                  140  
DEFAULT                  134  
user                     121  
TABLESPACE               99   
default                  98   
SYSTEM                   93   
VIEW                     91   
database                 88   
TEXT                     88   
SERVER                   55   
RULE                     51   
DOMAIN                   41   
EVENT                    32   
PROCEDURE                32   
TRIGGER                  29   
view                     29   
schema                   18   
system                   13   
INSTANCE                 11   
trigger                  10   
domain                   8    
AGGREGAT

In [239]:
print(get_stmt_and_file_count(expected_one_of))

[########################################] | 100% Completed | 6.14 ss
[########################################] | 100% Completed | 8.27 ss
(39405, 5965)


#### SEQUENCE

In [243]:
seq_chk = expected_one_of[
    (expected_one_of.found == 'SEQUENCE') &
    (expected_one_of.original.str.contains("ALTER SEQUENCE",regex=False,case=False))
]
print(get_stmt_and_file_count(seq_chk))

[########################################] | 100% Completed | 9.98 ss
[########################################] | 100% Completed | 8.02 ss
(32571, 3535)


In [241]:
seq_chk.head()

[########################################] | 100% Completed | 285.03 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
294,014818,13,"Query parsing failed.\n\tsql parser error: Expected one of TABLE or INDEX, found: SEQUENCE","one of TABLE or INDEX,",SEQUENCE,None,None,--\n-- Name: smtp_transaction_recipients_smtp_transaction_recipient_id_seq; Type: SEQUENCE OWNED BY; Schema: public; Owner: mipanel\n--\n\nALTER SEQUENCE smtp_transaction_recipients_smtp_transaction_recipient_id_seq OWNED BY smtp_transaction_recipients.smtp_transaction_recipient_id;
298,014818,18,"Query parsing failed.\n\tsql parser error: Expected one of TABLE or INDEX, found: SEQUENCE","one of TABLE or INDEX,",SEQUENCE,None,None,--\n-- Name: smtp_transactions_smtp_transaction_id_seq; Type: SEQUENCE OWNED BY; Schema: public; Owner: mipanel\n--\n\nALTER SEQUENCE smtp_transactions_smtp_transaction_id_seq OWNED BY smtp_transactions.smtp_transaction_id;
847,014875,52,"Query parsing failed.\n\tsql parser error: Expected one of TABLE or INDEX, found: SEQUENCE","one of TABLE or INDEX,",SEQUENCE,None,None,--\n-- Name: processing_studies_processing_studies_id_seq; Type: SEQUENCE OWNED BY; Schema: public; Owner: seqware\n--\n\nALTER SEQUENCE processing_studies_processing_studies_id_seq OWNED BY processing_studies.processing_studies_id;
851,014875,63,"Query parsing failed.\n\tsql parser error: Expected one of TABLE or INDEX, found: SEQUENCE","one of TABLE or INDEX,",SEQUENCE,None,None,--\n-- Name: file_type_file_type_id_seq; Type: SEQUENCE OWNED BY; Schema: public; Owner: seqware\n--\n\nALTER SEQUENCE file_type_file_type_id_seq OWNED BY file_type.file_type_id;
855,014875,74,"Query parsing failed.\n\tsql parser error: Expected one of TABLE or INDEX, found: SEQUENCE","one of TABLE or INDEX,",SEQUENCE,None,None,--\n-- Name: processing_experiments_processing_experiments_id_seq; Type: SEQUENCE OWNED BY; Schema: public; Owner: seqware\n--\n\nALTER SEQUENCE processing_experiments_processing_experiments_id_seq OWNED BY processing_experiments.processing_experiments_id;


#### final

In [244]:
no_seq = expected_one_of[
    ~((expected_one_of.found == 'SEQUENCE') &
    (expected_one_of.original.str.contains("ALTER SEQUENCE",regex=False,case=False)))
]

In [245]:
print(get_stmt_and_file_count(no_seq))

[########################################] | 100% Completed | 9.53 ss
[########################################] | 100% Completed | 7.99 ss
(6834, 2989)


In [294]:
to_write_down_hard.append(no_seq.compute(scheduler='processes'))

[########################################] | 100% Completed | 6.96 ss


### Expected "identifier,"

#### WHAT

In [246]:
expected_ident = expected_only[expected_only.expected == "identifier,"]
expected_ident['found'].value_counts().compute(scheduler='processes')

[########################################] | 100% Completed | 7.94 ss


(                                                                                                                                                                                                                                                                                                                                                16282
*                                                                                                                                                                                                                                                                                                                                                7184 
{                                                                                                                                                                                                                                                                                                                         

In [247]:
print(get_stmt_and_file_count(expected_ident))

[########################################] | 100% Completed | 6.92 ss
[########################################] | 100% Completed | 7.72 ss
(38696, 8816)


#### (

In [251]:
prnt_chk = expected_ident[
    (expected_ident.found == '(') &
    (expected_ident.original.str.contains("ALTER",case=False,regex=False)) &
    (expected_ident.original.str.contains("ADD",case=False,regex=False))
]
print(get_stmt_and_file_count(prnt_chk))

[########################################] | 100% Completed | 7.75 ss
[########################################] | 100% Completed | 8.38 ss
(15269, 2021)


In [249]:
prnt_chk.head()

[########################################] | 100% Completed | 246.99 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
172,014790,0,"Query parsing failed.\n\tsql parser error: Expected identifier, found: (","identifier,",(,None,None,alter table ACT_RE_PROCDEF add (ENGINE_VERSION_ NVARCHAR2(255));
173,014790,2,"Query parsing failed.\n\tsql parser error: Expected identifier, found: (","identifier,",(,None,None,alter table ACT_RE_DEPLOYMENT add (ENGINE_VERSION_ NVARCHAR2(255));
174,014790,4,"Query parsing failed.\n\tsql parser error: Expected identifier, found: (","identifier,",(,None,None,alter table ACT_RU_EXECUTION add (ROOT_PROC_INST_ID_ NVARCHAR2(64));
175,014790,6,"Query parsing failed.\n\tsql parser error: Expected identifier, found: (","identifier,",(,None,None,"alter table ACT_RU_EXECUTION add (IS_MI_ROOT_ NUMBER(1,0) CHECK (IS_MI_ROOT_ IN (1,0)));"
2284,015054,13,"Query parsing failed.\n\tsql parser error: Expected identifier, found: (","identifier,",(,None,None,--칼럼 loc칼럼추가\nalter table dept7 add (loc varchar2(14));


#### *

In [254]:
no_parant_ident = expected_ident[
    ~((expected_ident.found == '(') &
    (expected_ident.original.str.contains("ALTER",case=False,regex=False)) &
    (expected_ident.original.str.contains("ADD",case=False,regex=False)))
]
star_chk = no_parant_ident[
    (no_parant_ident.found == '*') &
    (expected_ident.original.str.contains("grant",case=False,regex=False))
]
print(get_stmt_and_file_count(star_chk))

[########################################] | 100% Completed | 7.48 ss
[########################################] | 100% Completed | 7.82 ss
(6853, 4759)


In [253]:
star_chk.head()

[########################################] | 100% Completed | 221.32 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
199,014802,2,"Query parsing failed.\n\tsql parser error: Expected identifier, found: *","identifier,",*,None,None,"GRANT SELECT, INSERT, DELETE ON `phpmyadmin`.* TO\n 'pma'@localhost;"
289,014816,1,"Query parsing failed.\n\tsql parser error: Expected identifier, found: *","identifier,",*,None,None,GRANT ALL PRIVILEGES ON proyecto_tic1v2.* TO 'grupo10_tic_2020'@'localhost';
320,014826,3,"Query parsing failed.\n\tsql parser error: Expected identifier, found: *","identifier,",*,None,None,"grant select, insert, delete, update on loja.* to user@'localhost';"
1654,014953,5,"Query parsing failed.\n\tsql parser error: Expected identifier, found: *","identifier,",*,None,None,GRANT ALL on camundabpmn.* to 'camunda' identified by 'camunda123' with GRANT OPTION;
2102,015022,14,"Query parsing failed.\n\tsql parser error: Expected identifier, found: *","identifier,",*,None,None,"GRANT SELECT, INSERT, TRIGGER, UPDATE, DELETE ON TABLE * TO 'todos'@'localhost';"


#### final

In [255]:
no_star_idn = no_parant_ident[
    ~((no_parant_ident.found == '*') &
    (expected_ident.original.str.contains("grant",case=False,regex=False)))
]

In [256]:
print(get_stmt_and_file_count(no_star_idn))

[########################################] | 100% Completed | 8.60 ss
[########################################] | 100% Completed | 7.77 ss
(16574, 2191)


In [295]:
to_write_down_hard.append(no_star_idn.compute(scheduler='processes'))

[########################################] | 100% Completed | 7.02 ss


### TODO? Expected "PRIMARY, UNIQUE, FOREIGN, or CHECK,"

#### WHAT

In [257]:
expected_pk_uk_f = expected_only[expected_only.expected == "PRIMARY, UNIQUE, FOREIGN, or CHECK,"]
expected_pk_uk_f['found'].value_counts().compute(scheduler='processes')

[########################################] | 100% Completed | 7.68 ss


key                                           4068
.                                             2638
KEY                                           2444
(                                             355 
DEFAULT                                       254 
NOT                                           228 
default                                       56  
EXCLUDE                                       35  
CLUSTER                                       18  
`i_op_dc_vnet_alloc__vnet__data_center_id`    17  
index                                         11  
Key                                           10  
FOREING                                       10  
,                                             9   
not                                           8   
%                                             6   
Default                                       6   
_pkey                                         6   
-                                             5   
exclude                        

In [258]:
print(get_stmt_and_file_count(expected_pk_uk_f))

[########################################] | 100% Completed | 6.96 ss
[########################################] | 100% Completed | 8.28 ss
(10344, 1489)


#### KEY

In [260]:
k_chk = expected_pk_uk_f[
    (expected_pk_uk_f.found.str.contains("key",case=False,regex=False))
]

In [261]:
k_chk.head()

[########################################] | 100% Completed | 261.46 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
27611,017230,0,"Query parsing failed.\n\tsql parser error: Expected PRIMARY, UNIQUE, FOREIGN, or CHECK, found: KEY","PRIMARY, UNIQUE, FOREIGN, or CHECK,",KEY,None,None,"CREATE TABLE EMPLOYEE\n(\n Empno VARCHAR(5) PRIMARY KEY,\n Ename VARCHAR(20) UNIQUE,\n Designation VARCHAR(20) NOT NULL,\n Salary FLOAT(10) DEFAULT 25000,\n DOB DATE NOT NULL,\n Dno FLOAT(3),\n CONSTRAINT FOREIGN KEY (Dno) REFERENCES DEPARTMENT(Deptno),\n CONSTRAINT CHECK (Salary BETWEEN 15000 AND 50000),\n CONSTRAINT CHECK (Empno LIKE ""E%"")\n);"
28533,017317,2,"Query parsing failed.\n\tsql parser error: Expected PRIMARY, UNIQUE, FOREIGN, or CHECK, found: KEY","PRIMARY, UNIQUE, FOREIGN, or CHECK,",KEY,None,None,ALTER TABLE chapter_t ADD CONSTRAINT FOREIGN KEY (book_id) REFERENCES book_t (book_id);
28784,017347,11,"Query parsing failed.\n\tsql parser error: Expected PRIMARY, UNIQUE, FOREIGN, or CHECK, found: key","PRIMARY, UNIQUE, FOREIGN, or CHECK,",key,None,None,alter table CONN_SERV_REG\n add constraint foreign key (CONN_ROUTE_PLAT_CO)\n references CONN_SRP_REG (CONN_ROUTE_PLAT_CO) \n constraint FK_REFEREN_CONN_SR;
28785,017347,12,"Query parsing failed.\n\tsql parser error: Expected PRIMARY, UNIQUE, FOREIGN, or CHECK, found: key","PRIMARY, UNIQUE, FOREIGN, or CHECK,",key,None,None,alter table CONN_SERV_REG\n add constraint foreign key (WSDL_ID)\n references CONN_WSDL (WSDL_ID) \n constraint FK_REFEREN_CONN_WS;
28786,017347,13,"Query parsing failed.\n\tsql parser error: Expected PRIMARY, UNIQUE, FOREIGN, or CHECK, found: key","PRIMARY, UNIQUE, FOREIGN, or CHECK,",key,None,None,alter table CONN_SERV_REG\n add constraint foreign key (PROTOCOL_ID)\n references CONN_PROTOCOL (PROTOCOL_ID) \n constraint FK_REFEREN_CONN_PR;


#### dot

#### final

In [296]:
to_write_down_hard.append(expected_pk_uk_f.compute(scheduler='processes'))

[########################################] | 100% Completed | 6.97 ss


### Expected "ON,"

In [262]:
expected_on = expected_only[expected_only.expected == "ON,"]
expected_on['found'].value_counts().compute(scheduler='processes')


[########################################] | 100% Completed | 7.36 ss


USING           2786
"transfers"     354 
(               329 
using           221 
public          218 
               ...  
file8_pk_idx    1   
file9_pk_idx    1   
"photos"        1   
"phantom2"      1   
instance        1   
Name: found, Length: 1207, dtype: int64

In [263]:
print(get_stmt_and_file_count(expected_on))

[########################################] | 100% Completed | 6.36 ss
[########################################] | 100% Completed | 6.49 ss
(8468, 1387)


#### USING

In [266]:
using_check_on = expected_on[
    (expected_on.found == 'USING') &
    (expected_on.original.str.contains("INDEX",case=False,regex=False))
]
print(get_stmt_and_file_count(using_check_on))

[########################################] | 100% Completed | 7.01 ss
[########################################] | 100% Completed | 6.56 ss
(2786, 190)


In [265]:
using_check_on.head()

[########################################] | 100% Completed | 226.45 ms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
30332,017541,21,"Query parsing failed.\n\tsql parser error: Expected ON, found: USING","ON,",USING,None,None,-- -------------------------------------------------------------\n\n\n-- CREATE INDEX 'idx_kreator_asocijacija' ----------------------\nCREATE INDEX `idx_kreator_asocijacija` USING BTREE ON `asocijacija`( `kreator_id` );
30333,017541,22,"Query parsing failed.\n\tsql parser error: Expected ON, found: USING","ON,",USING,None,None,-- -------------------------------------------------------------\n\n\n\n-- CREATE INDEX 'fk_sablon_igre_igra' --------------------------\nCREATE INDEX `fk_sablon_igre_igra` USING BTREE ON `igra`( `sablon_igre_id` );
30334,017541,23,"Query parsing failed.\n\tsql parser error: Expected ON, found: USING","ON,",USING,None,None,"-- -------------------------------------------------------------\n\n\n-- CREATE INDEX 'idx_broj_igranja_igra' ------------------------\nCREATE INDEX `idx_broj_igranja_igra` USING BTREE ON `igra`( `broj_igranja`, `aktivna` );"
30335,017541,24,"Query parsing failed.\n\tsql parser error: Expected ON, found: USING","ON,",USING,None,None,"-- -------------------------------------------------------------\n\n\n-- CREATE INDEX 'idx_kategorija_igra' --------------------------\nCREATE INDEX `idx_kategorija_igra` USING BTREE ON `igra`( `kategorija_id`, `aktivna` );"
30336,017541,25,"Query parsing failed.\n\tsql parser error: Expected ON, found: USING","ON,",USING,None,None,"-- -------------------------------------------------------------\n\n\n-- CREATE INDEX 'idx_kreator_igra' -----------------------------\nCREATE INDEX `idx_kreator_igra` USING BTREE ON `igra`( `kreator_id`, `aktivna` );"


#### final

In [267]:
no_using_on = expected_on[
    ~((expected_on.found == 'USING') &
    (expected_on.original.str.contains("INDEX",case=False,regex=False)))
]

In [268]:
print(get_stmt_and_file_count(no_using_on))

[########################################] | 100% Completed | 7.75 ss
[########################################] | 100% Completed | 6.59 ss
(5682, 1201)


In [297]:
to_write_down_hard.append(no_using_on.compute(scheduler='processes'))

[########################################] | 100% Completed | 7.55 ss


### Expected "SET/DROP NOT NULL, SET DEFAULT, SET DATA TYPE after ALTER COLUMN,"

#### WHAT

In [269]:
expected_alter_column = expected_only[expected_only.expected == "SET/DROP NOT NULL, SET DEFAULT, SET DATA TYPE after ALTER COLUMN,"]
expected_alter_column['found'].value_counts().compute(scheduler='processes')


[########################################] | 100% Completed | 7.98 ss


ADD                                       1159
SET                                       415 
NOT                                       353 
VARCHAR                                   344 
position                                  303 
varchar                                   245 
NVARCHAR                                  207 
TYPE                                      189 
BIGINT                                    148 
POSITION                                  146 
bigint                                    104 
RENAME                                    101 
INT                                       99  
nvarchar                                  95  
rename                                    94  
DEFAULT                                   78  
integer                                   76  
int                                       72  
set                                       65  
IDENTITY                                  56  
tinyint                                   56  
FLOAT        

In [271]:
print(get_stmt_and_file_count(expected_alter_column))

[########################################] | 100% Completed | 7.00 ss
[########################################] | 100% Completed | 7.13 ss
(5006, 965)


#### ADD

In [275]:
alter_col_add_chk = expected_alter_column[
    (expected_alter_column.found == 'ADD') &
    (expected_alter_column.original.str.contains("IDENTITY",case=False,regex=False))
]
print(get_stmt_and_file_count(alter_col_add_chk))

[########################################] | 100% Completed | 8.11 ss
[########################################] | 100% Completed | 6.67 ss
(1159, 238)


In [273]:
alter_col_add_chk.head()

[########################################] | 100% Completed | 238.04 ms


c:\Users\geacu\anaconda3\envs\windows\lib\site-packages\dask\dataframe\core.py:7893: UserWarning: Insufficient elements for `head`. 5 elements requested, only 2 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
37517,018214,17,"Query parsing failed.\n\tsql parser error: Expected SET/DROP NOT NULL, SET DEFAULT, SET DATA TYPE after ALTER COLUMN, found: ADD","SET/DROP NOT NULL, SET DEFAULT, SET DATA TYPE after ALTER COLUMN,",ADD,None,None,--\n-- TOC entry 204 (class 1259 OID 16756)\n-- Name: files_id_seq; Type: SEQUENCE; Schema: public; Owner: root\n--\n\nALTER TABLE public.files ALTER COLUMN id ADD GENERATED ALWAYS AS IDENTITY (\n SEQUENCE NAME public.files_id_seq\n START WITH 1\n INCREMENT BY 1\n NO MINVALUE\n NO MAXVALUE\n CACHE 1\n);
37519,018214,20,"Query parsing failed.\n\tsql parser error: Expected SET/DROP NOT NULL, SET DEFAULT, SET DATA TYPE after ALTER COLUMN, found: ADD","SET/DROP NOT NULL, SET DEFAULT, SET DATA TYPE after ALTER COLUMN,",ADD,None,None,--\n-- TOC entry 202 (class 1259 OID 16747)\n-- Name: users_id_seq; Type: SEQUENCE; Schema: public; Owner: root\n--\n\nALTER TABLE public.users ALTER COLUMN id ADD GENERATED ALWAYS AS IDENTITY (\n SEQUENCE NAME public.users_id_seq\n START WITH 1\n INCREMENT BY 1\n NO MINVALUE\n NO MAXVALUE\n CACHE 1\n);


[########################################] | 100% Completed | 6.78 ss
[########################################] | 100% Completed | 7.05 ss
(5006, 965)


#### SET

In [279]:
no_add_alter_col = expected_alter_column[
    ~((expected_alter_column.found == 'ADD') &
    (expected_alter_column.original.str.contains("IDENTITY",case=False,regex=False)))
]

set_chk_alter_col = no_add_alter_col[
    (no_add_alter_col.found == 'SET')
]
print(get_stmt_and_file_count(set_chk_alter_col))

[########################################] | 100% Completed | 7.31 ss
[########################################] | 100% Completed | 6.55 ss
(415, 72)


In [278]:
set_chk_alter_col.head(npartitions=60)

[########################################] | 100% Completed | 4.29 sms


,file_id,statement_id,parse_error,expected,found,unterminated_what,unterminated_where,original
34905,017991,19,"Query parsing failed.\n\tsql parser error: Expected SET/DROP NOT NULL, SET DEFAULT, SET DATA TYPE after ALTER COLUMN, found: SET","SET/DROP NOT NULL, SET DEFAULT, SET DATA TYPE after ALTER COLUMN,",SET,None,None,ALTER TABLE ONLY sm_atp_workflow_runs ALTER COLUMN workflow_run_id SET STATISTICS 0;
41180,018671,0,"Query parsing failed.\n\tsql parser error: Expected SET/DROP NOT NULL, SET DEFAULT, SET DATA TYPE after ALTER COLUMN, found: SET","SET/DROP NOT NULL, SET DEFAULT, SET DATA TYPE after ALTER COLUMN,",SET,None,None,"ALTER TABLE items_applications ALTER COLUMN itemappid SET WITH DEFAULT NULL\n/\nREORG TABLE items_applications\n/\nALTER TABLE items_applications ALTER COLUMN applicationid SET WITH DEFAULT NULL\n/\nREORG TABLE items_applications\n/\nALTER TABLE items_applications ALTER COLUMN itemid SET WITH DEFAULT NULL\n/\nREORG TABLE items_applications\n/\nDROP INDEX items_applications_1\n/\nDELETE FROM items_applications WHERE applicationid NOT IN (SELECT applicationid FROM applications)\n/\nDELETE FROM items_applications WHERE itemid NOT IN (SELECT itemid FROM items)\n/\nCREATE UNIQUE INDEX items_applications_1 ON items_applications (applicationid,itemid)\n/\nALTER TABLE items_applications ADD CONSTRAINT c_items_applications_1 FOREIGN KEY (applicationid) REFERENCES applications (applicationid) ON DELETE CASCADE\n/\nALTER TABLE items_applications ADD CONSTRAINT c_items_applications_2 FOREIGN KEY (itemid) REFERENCES items (itemid) ON DELETE CASCADE\n/"
3222,027401,0,"Query parsing failed.\n\tsql parser error: Expected SET/DROP NOT NULL, SET DEFAULT, SET DATA TYPE after ALTER COLUMN, found: SET","SET/DROP NOT NULL, SET DEFAULT, SET DATA TYPE after ALTER COLUMN,",SET,None,None,"ALTER TABLE users_groups ALTER COLUMN id SET WITH DEFAULT NULL\n/\nREORG TABLE users_groups\n/\nALTER TABLE users_groups ALTER COLUMN usrgrpid SET WITH DEFAULT NULL\n/\nREORG TABLE users_groups\n/\nALTER TABLE users_groups ALTER COLUMN userid SET WITH DEFAULT NULL\n/\nREORG TABLE users_groups\n/\nDROP INDEX users_groups_1\n/\nDELETE FROM users_groups WHERE usrgrpid NOT IN (SELECT usrgrpid FROM usrgrp)\n/\nDELETE FROM users_groups WHERE userid NOT IN (SELECT userid FROM users)\n/\nCREATE UNIQUE INDEX users_groups_1 ON users_groups (usrgrpid,userid)\n/\nALTER TABLE users_groups ADD CONSTRAINT c_users_groups_1 FOREIGN KEY (usrgrpid) REFERENCES usrgrp (usrgrpid) ON DELETE CASCADE\n/\nALTER TABLE users_groups ADD CONSTRAINT c_users_groups_2 FOREIGN KEY (userid) REFERENCES users (userid) ON DELETE CASCADE\n/"
23014,025774,9,"Query parsing failed.\n\tsql parser error: Expected SET/DROP NOT NULL, SET DEFAULT, SET DATA TYPE after ALTER COLUMN, found: SET","SET/DROP NOT NULL, SET DEFAULT, SET DATA TYPE after ALTER COLUMN,",SET,None,None,ALTER TABLE ONLY content ALTER COLUMN erdf SET STORAGE MAIN;
23015,025774,10,"Query parsing failed.\n\tsql parser error: Expected SET/DROP NOT NULL, SET DEFAULT, SET DATA TYPE after ALTER COLUMN, found: SET","SET/DROP NOT NULL, SET DEFAULT, SET DATA TYPE after ALTER COLUMN,",SET,None,None,ALTER TABLE ONLY content ALTER COLUMN svg SET STORAGE MAIN;


#### final

In [281]:
no_set_alter_col = no_add_alter_col[
    ~(no_add_alter_col.found == 'SET')
]
print(get_stmt_and_file_count(no_set_alter_col))

[########################################] | 100% Completed | 7.96 ss
[########################################] | 100% Completed | 6.96 ss
(3432, 670)


In [298]:
to_write_down_hard.append(no_set_alter_col.compute(scheduler='processes'))

[########################################] | 100% Completed | 7.02 ss


## Figure out how to retain only unparsed statements and write them down to parquet + write down unparsable files to csv

### Concat and write stmts

In [299]:
to_write_df = pd.concat(to_write_down_hard)

In [300]:
ddf = dd.from_pandas(to_write_df, npartitions=60)

In [301]:
ddf.to_parquet('../out_new/rustparser_details_unparsed_stmts_only_hard_left_unparsed/',schema=not_parsed_schema,overwrite=True)

[########################################] | 100% Completed | 2.32 sms


### Read stmts

In [302]:
unparsed1 = dd.read_parquet('../out_new/rustparser_details_unparsed_stmts_only_hard_left_unparsed/', columns = not_parsed_schema.names ,schema=not_parsed_schema, split_row_groups=True, calculate_divisions=True,engine='pyarrow')

In [303]:
print(get_stmt_and_file_count(unparsed1))

[########################################] | 100% Completed | 3.22 ss
[########################################] | 100% Completed | 3.22 ss
(1074829, 129133)


### Read hard to parse ids

In [304]:
difficult_to_parse_file_ids = unparsed1['file_id'].unique().compute(scheduler='processes')

[########################################] | 100% Completed | 3.93 ss


In [305]:
difficult_to_parse_file_ids.to_csv('rustparser_files_left_difficult_to_parse.csv',header=True,index=False)